# Deep Value-based Reinforcement Learning

<img src="https://github.com/jeremiedecock/polytechnique-inf639-2024-students/blob/main/assets/logo.jpg?raw=true" style="float: left; width: 15%" />

[CSC_53439_EP-2024](https://moodle.polytechnique.fr/course/view.php?id=19358) Lab session #1

2019-2024 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-inf639-2024-students/blob/main/lab1_deep_value-based_reinforcement_learning.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf639-2024-students/main?filepath=lab1_deep_value-based_reinforcement_learning.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-inf639-2024-students/blob/main/lab1_deep_value-based_reinforcement_learning.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-inf639-2024-students/raw/main/lab1_deep_value-based_reinforcement_learning.ipynb)

## Introduction

The aim of this lab is to provide an in-depth exploration of the most renowned value-based reinforcement learning techniques, specifically *Deep Q-Networks* and its enhancements.

In this Python notebook, you will implement and evaluate *Deep Q-Networks* (DQN) and its various adaptations.

You can either:
- open, edit and execute the notebook in *Google Colab* following this link: https://colab.research.google.com/github/jeremiedecock/polytechnique-inf639-2024-students/blob/main/lab1_deep_value-based_reinforcement_learning.ipynb ; this is the **recommended** choice as you have nothing to install on your computer
- open, edit and execute the notebook in *MyBinder* (if for any reason the Google Colab solution doesn't work): https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf639-2024-students/main?filepath=lab1_deep_value-based_reinforcement_learning.ipynb
- download, edit and execute the notebook on your computer if Python3 and JypyterLab are already installed: https://github.com/jeremiedecock/polytechnique-inf639-2024-students/raw/main/lab1_deep_value-based_reinforcement_learning.ipynb

If you work with Google Colab or MyBinder, **remember to save or download your work regularly or you may lose it!**

### Important note

This tutorial has been tested with Python 3.10 and Python 3.11.

It is important to note that **Bonus Section 4: *Test and train a DQN agent to play Atari games*** [is not compatible with Python 3.12](https://github.com/Farama-Foundation/Gymnasium/issues/1081) (the latest stable version of Python). If you plan to complete this bonus section locally on your machine rather than on Google Colab, make sure you have Python 3.10 or 3.11 installed.

If you are using Python 3.12 and prefer not to run this notebook on Google Colab, an alternative solution via Docker will be provided at the beginning of Bonus Section 4.

### Survey

Please answer the following survey to help us improve this lab session: https://moodle.polytechnique.fr/mod/questionnaire/view.php?id=535310

### Deep value-based reinforcement learning

Deep reinforcement learning methods like DQN (Deep Q-Networks) are significant advancements over tabular methods such as Q-Learning because they can handle complex, high-dimensional environments that were previously intractable. While Q-Learning is limited to environments where the state and action spaces are sufficiently small to maintain a table of values, DQN uses neural networks to approximate the Q-value function, allowing it to generalize across similar states and scale to problems with vast state spaces. This enables DQN to learn optimal policies for tasks like video games, robotic control, and other applications where the number of possible states is extraordinarily large.

While DQN was designed to tackle large environments like Atari games, the primary focus of this lab is to delve into the underlying algorithms, understand them thoroughly, and evaluate them comprehensively. It's important to note that working with not-so-deep networks captures the essence of deep reinforcement learning, excluding the computational expense. The transition from tabular Q-learning to DQN involves significant implications, primarily due to the ability of DQN to handle high-dimensional state spaces. Moving from DQN to very-deep-DQN is primarily a matter of scale and computational resources. The core principles remain the same, and understanding these principles is the key to mastering reinforcement learning, regardless of the complexity of the network used.
For these reasons, in this lab, we will focus on studying the CartPole environment. The CartPole problem is a classic in reinforcement learning, and it provides a simpler and more manageable context for understanding the principles of DQN. The convergence in the CartPole environment is much faster than in Atari games - typically within a minute, as opposed to approximately hours on a well-equipped personal computer for Atari games. This allows us to experiment and iterate more quickly, facilitating a deeper understanding of the algorithms at play.

We will therefore focus on algorithms in a simple environment. However, once the key concepts are mastered and tested in these settings, Bonus Section 4 will offer you the opportunity to refine your technical skills by applying them to the game Breakout in the Atari environment.

## Setup the Python environment

This notebook relies on several libraries including `torch`, `gymnasium`, `numpy`, `pandas`, `seaborn`, `imageio`, `pygame`, and `tqdm`.
A complete list of dependencies can be found in the provided [requirements-minimal.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements-minimal.txt) and [requirements.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements.txt) files.

- [requirements-minimal.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements.txt) contains the minimal dependencies required to run this notebook without the optional sections (e.g. the Atari environment, Aim, Tensorboard, Optuna, Weights&Bias, ...).
- [requirements.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements.txt) contains all the dependencies required to run this notebook with all the optional sections.

### If you use Google Colab

If you use Google Colab, execute the next cell to install required libraries.

In [ ]:
import sys, subprocess

def is_colab():
    return "google.colab" in sys.modules

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())

if is_colab():
    run_subprocess_command("apt install xvfb x11-utils")
    run_subprocess_command("pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements-google-colab.txt")

In [ ]:
#! apt install xvfb x11-utils && pip install gymnasium pyvirtualdisplay

### If you have downloaded the notebook on your computer and execute it in your own Python environment

To set up the necessary dependencies, first download the [requirements.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements.txt) or [requirements-minimal.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf639-2024-students/main/requirements-minimal.txt) depending on whether you want to run the optional sections of this notebook or not (c.f. *Setup the Python environment* section above).

Ensure it is located in the same directory as this notebook. Next, run the following command to establish a [Python virtual environment (venv)](https://docs.python.org/3/library/venv.html) that includes all the essential libraries for this lab.

#### On Posix systems (Linux, MacOSX, WSL, ...)

```bash
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

Adapt the name of the requirements file if you have chosen to use the minimal version.

#### On Windows

```bash
python3 -m venv env
env\Scripts\activate.bat
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

Adapt the name of the requirements file if you have chosen to use the minimal version.

### Run notebooks locally in a dedicated Docker container

If you are familiar with Docker, an image is available on Docker Hub for this lab:

```bash
docker run -it --rm -p 8888:8888 -v "${PWD}":/home/jovyan/work jdhp/inf639-lab1:latest
```

If you encounter an error during the notebook's execution indicating that writing a file is not possible, this issue may stem from the user ID within the container lacking the necessary permissions in the project directory. This problem can be resolved by modifying the directory's permissions, for example, using the command:

```bash
chmod 777 . figs models
rm -rf figs/*.gif
rm -rf figs/*.png
rm -rf models/*.pth
```

### Import required packages

In [ ]:
import collections
import gymnasium as gym
import itertools
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from pathlib import Path
import random
import time
import torch
from torch.optim.lr_scheduler import _LRScheduler
from typing import List, Tuple, Deque, Optional, Callable
# from inf639 import *

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
gym.__version__

In [ ]:
sns.set_context("talk")

In [ ]:
FIGS_DIR = Path("figs/")       # Where to save figures (.gif files)
MODELS_DIR = Path("models/")   # Where to save models (.pth files)

In [ ]:
if not FIGS_DIR.exists():
    FIGS_DIR.mkdir()
if not MODELS_DIR.exists():
    MODELS_DIR.mkdir()

### Create a Gymnasium rendering wrapper to visualize environments as GIF images within the notebook

This notebook allows you to visualize the episodes as animated GIFs. Run the cell below to enable this feature.

In [ ]:
# To display GIF images in the notebook

import imageio     # To render episodes in GIF images (otherwise there would be no render on Google Colab)
                   # C.f. https://stable-baselines.readthedocs.io/en/master/guide/examples.html#bonus-make-a-gif-of-a-trained-agent
import IPython
from IPython.display import Image

if is_colab():
    import pyvirtualdisplay

    _display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                        size=(1400, 900))
    _ = _display.start()

class RenderWrapper:
    def __init__(self, env, force_gif=False):
        self.env = env
        self.force_gif = force_gif
        self.reset()

    def reset(self):
        self.images = []

    def render(self):
        if not is_colab():
            self.env.render()
            time.sleep(1./60.)

        if is_colab() or self.force_gif:
            img = self.env.render()         # Assumes env.render_mode == 'rgb_array'
            self.images.append(img)

    def make_gif(self, filename="render"):
        if is_colab() or self.force_gif:
            gif_path = filename.with_suffix('.gif')
            imageio.mimsave(gif_path, [np.array(img) for i, img in enumerate(self.images) if i%2 == 0], fps=29, loop=0)
            return Image(open(gif_path,'rb').read())

    @classmethod
    def register(cls, env, force_gif=False):
        env.render_wrapper = cls(env, force_gif=True)

## Define some parameters

### Number of trainings

To achieve more representative outcomes at the conclusion of each exercise, we average the results across multiple training sessions. The `NUMBER_OF_TRAININGS` variable specifies the number of training sessions conducted before the results are displayed. 

We recommend setting a lower value (such as 1 or 2) during the development and testing phases of your implementations. Once you have completed your work and are confident in its functionality, you can increase the number of training sessions to minimize the variance in results. Be aware that a higher number of training sessions will extend the execution time, so adjust this setting in accordance with your computer's capabilities.

Additionally, you have the option to assign a specific value to the `NUMBER_OF_TRAININGS` variable for each exercise directly within the cells where the training loop is defined (the `NUMBER_OF_TRAININGS` variable is commented out at the beginning of these cells).

In [ ]:
DEFAULT_NUMBER_OF_TRAININGS = 3

## PyTorch Refresher and Cheat Sheet

In this lab, we will be implementing our deep reinforcement learning algorithms using PyTorch.
If you need a refresher, you might find this [PyTorch Cheat Sheet](https://pytorch.org/tutorials/beginner/ptcheat.html) helpful. It provides a quick reference for many of the most commonly used PyTorch functions and concepts, and can be a valuable resource as you work through this lab.

You can also refer to the [official documentation](https://pytorch.org/docs/stable/index.html).

### PyTorch device

PyTorch can run on both CPUs and GPUs. The following cell will determine the device PyTorch will use. If a GPU is available, PyTorch will use it; otherwise, it will use the CPU.

For utilizing a GPU on Google Colab, you also have to activate it following the steps outlined [here](https://colab.research.google.com/notebooks/gpu.ipynb).

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set the device to CUDA if available, otherwise use CPU

Note that the GPU is not very useful for CartPole (but useful for Atari Breakout in *Bonus section 4*) because CartPole is a simple and quick problem to solve, and CUDA spends more time transferring data between the CPU and GPU than processing it directly on the CPU.

You can uncomment the next cell to explicitly instruct PyTorch to train neural networks using the CPU.

In [ ]:
# device = "cpu"

In [ ]:
print(f"PyTorch will train and test neural networks on {device}")

In [ ]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")

If you have a recent GPU (e.g. RTX 4060 Ti 16G) and want to use it, you may need to install a specific version of PyTorch compatible with your Cuda version (e.g. Cuda 12.4). For this, you will have to edit the `requirements.txt` file and replace the current version of PyTorch with the one compatible with your Cuda version. Check the [official PyTorch website](https://pytorch.org/get-started/locally/) for more information.

## Bonus section 1: Monitoring the training process with Tensorboard and Aim

Monitoring the training process is crucial for understanding the agent's learning progress. In this section, we will introduce two open-source tools that can help you visualize the training process: Tensorboard and Aim.

A third tool, [Weights & Biases (W&B)](https://wandb.ai/site) will be presented in *Bonus section 3*.

Tensorflow and Aim are particularly useful for monitoring the training process on a local machine, as they do not require a server to run.
On the opposite, W&B is a cloud-based solution that allows you to monitor the training process from anywhere.
Other solutions will be covered in the next tutorials.

### Monitoring the training process with Tensorboard

TensorBoard is a visualization tool developed by Google, primarily used to track and visualize metrics during the training of deep learning models. While initially designed for TensorFlow, it can also be integrated with **PyTorch** through the **torch.utils.tensorboard** API. This allows real-time tracking of metrics such as losses, accuracy, computation graphs, and even examining images, weight distributions, histograms, and more.

The following cell demonstrates how to use TensorBoard with PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# Simple example: model and data
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Creating a model, optimizer, and loss function
model = SimpleModel()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Initializing TensorBoard SummaryWriter
writer = SummaryWriter()

# Training example
for epoch in range(100):
    # Fake data
    inputs = torch.randn(32, 10)
    targets = torch.randn(32, 1)
    
    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, targets)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Log the loss values to TensorBoard
    writer.add_scalar('Loss/train', loss.item(), epoch)

# Closing SummaryWriter
writer.close()

**`SummaryWriter`** is the central tool for logging data to TensorBoard. It captures metric values, like loss, for each iteration/epoch.

This example trains a simple model on fake data (32 samples with 10 features) over 100 epochs, and at each epoch, the loss value is sent to TensorBoard via `writer.add_scalar`.

To visualize the data from a local machine, run the following command in the terminal:

```bash
tensorboard --logdir=runs
```

then open a web browser and go at the address mentioned in the terminal (usually http://localhost:6006/).

If you are using Google Colab, you can use the **TensorBoard magic** to visualize the data directly in the notebook.

In [ ]:
# Load TensorBoard extension in Colab
%load_ext tensorboard
%tensorboard --logdir ./runs

For mor information, check the Official Documentation:
- [TensorBoard for PyTorch Documentation](https://pytorch.org/docs/stable/tensorboard.html)
- [Official TensorBoard Documentation (General)](https://www.tensorflow.org/tensorboard)

### Monitoring the training process with AIM

Aim is an open-source alternative to TensorBoard for managing and visualizing machine learning experiments.
Designed to be simpler and more flexible, **Aim** allows tracking and analyzing metrics, hyperparameters, model outputs, and more for projects using PyTorch, TensorFlow, or other frameworks.

<img src="https://user-images.githubusercontent.com/13848158/136374529-af267918-5dc6-4a4e-8ed2-f6333a332f96.gif" width="600px"></img>

The following cell demonstrates how to use Aim with PyTorch.

In [ ]:
import torch
import aim

# Initialize Aim
run = aim.Run()

# set training hyperparameters
run['hparams'] = {
    'learning_rate': 0.01,
    'batch_size': 32,
}

# Simple model example
model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    inputs = torch.randn(10, 1)
    targets = torch.randn(10, 1)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track the 'loss' metric
    run.track(loss.item(), name='loss', epoch=epoch)    # Log metric with Aim

This example trains a simple model on fake data over 10 epochs, and at each epoch, the loss value is sent to Aim via `run.track`.

To visualize the data from a local machine, run the following command in the terminal:

```bash
aim up
```

then open a web browser and go at the address mentioned in the terminal (usually http://localhost:43800/).

Or alternatively, you can use the **Aim magic** to [visualize the data directly in the notebook](https://aimstack.readthedocs.io/en/latest/using/jupyter_notebook_ui.html) (for instance if you are using Google Colab).

In [ ]:
%load_ext aim
%aim up

For more information, check the official Documentation:
- [Aim Documentation](https://aimstack.readthedocs.io/en/latest/)
- [GitHub Aim](https://github.com/aimhubio/aim)
- [Quick start](https://github.com/aimhubio/aim#-quick-start)
- [Getting started](https://aimstack.readthedocs.io/en/latest/quick_start/setup.html)

## Part 1: Hands on Cart Pole environment [reminder]

For the purpose of focusing on the algorithms, we will use standard environments provided by the Gymnasium suite.
Gymnasium provides controllable environments (https://gymnasium.farama.org/environments/classic_control/) for research in reinforcement learning.
Especially, we will try to solve the CartPole-v1 environment (c.f. https://gymnasium.farama.org/environments/classic_control/cart_pole/) which offers a continuous state space and discrete action space.
The Cart Pole task consists in maintaining a pole in a vertical position by moving a cart on which the pole is attached with a joint.
No friction is considered.
The task is supposed to be solved if the pole stays up-right (within 15 degrees) for 200 steps in average over 100 episodes while keeping the cart position within reasonable bounds.
The state is given by $\{x,\frac{\partial x}{\partial t},\omega,\frac{\partial \omega}{\partial t}\}$ where $x$ is the position of the cart and $\omega$ is the angle between the pole and vertical position.
There are only two possible actions: $a \in \{0, 1\}$ where $a = 0$ means "push the cart to the LEFT" and $a = 1$ means "push the cart to the RIGHT".

### Exercise 1: Hands on Cart Pole

**Task 1:** refer to the following link [CartPole Environment](https://gymnasium.farama.org/environments/classic_control/cart_pole/) to familiarize yourself with the CartPole environment if you are not already.

**Note:** for a refresher on the key concepts of Gymnasium, you can visit this [Basic Usage Guide](https://gymnasium.farama.org/content/basic_usage/).

Print some information about the environment:

In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n.item()

print(f"State space size is: { env.observation_space }")
print(f"Action space size is: { env.action_space }")
print("Actions are: {" + ", ".join([str(a) for a in range(env.action_space.n)]) + "}")

env.close()

**Task 2:** Run the following cells and check different basic 
policies (for instance constant actions or randomly drawn actions) to discover the CartPole environment.
Although this environment has easy dynamics that can be computed analytically, we will solve this problem with Policy Gradient based Reinforcement learning.

### Test the CartPole environment with a constant policy

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

observation, info = env.reset()
done = False

for t in range(50):
    env.render_wrapper.render()

    if not done:
        print(observation)
    else:
        print("x", end="")

    action = 0
    observation, reward, done, truncated, info = env.step(action)


print()
env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_ex1left")

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

observation, info = env.reset()
done = False

for t in range(50):
    env.render_wrapper.render()

    if not done:
        print(observation)
    else:
        print("x", end="")

    action = 1
    observation, reward, done, truncated, info = env.step(action)

print()
env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_ex1right")

### Test the CartPole environment with a random policy

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

for episode_index in range(5):
    observation, info = env.reset()
    done = False

    for t in range(70):
        env.render_wrapper.render()

        if not done:
            print(observation)
        else:
            print("x", end="")


        action = env.action_space.sample()
        observation, reward, done, truncated, info = env.step(action)

    print()
    env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_ex1random")

## Part 2: Deep value-based reinforcement learning with Deep Q-Networks (DQN) [reminder]

In this part, we will begin our exploration of deep reinforcement learning with Deep Q-Networks (DQN), a famous value-based method.

Deep reinforcement learning methods like DQN (Deep Q-Networks) are significant advancements over tabular methods such as Q-Learning because they can handle complex, high-dimensional environments that were previously intractable. While Q-Learning is limited to environments where the state and action spaces are sufficiently small to maintain a table of values, DQN uses neural networks to approximate the Q-value function, allowing it to generalize across similar states and scale to problems with vast state spaces. This enables DQN to learn optimal policies for tasks like video games, robotic control, and other applications where the number of possible states is extraordinarily large.

### Exercise 2: Implement a naive value-based deep reinforcement learning algorithm

Our first step will be to write a naive implementation of a version of Q-Learning, where the Q-function is approximated by a neural network. This approach combines traditional Q-Learning with the power of function approximation provided by neural networks, allowing us to handle environments with large state spaces.

#### The algorithm

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\mathbf{s} \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $\mathbf{a} \leftarrow \epsilon\text{-greedy}(\mathbf{s}, \hat{Q}_{\mathbf{\omega}}$ $)$<br>
		$\quad\quad$ $r, \mathbf{s'} \leftarrow \text{env.step}(\mathbf{a})$<br>
		$\quad\quad$ $\mathbf{\omega} \leftarrow \mathbf{\omega} + \alpha \left[ r + \gamma \max_{\mathbf{a}^\star \in \mathcal{A}}\hat{Q}_{\mathbf{\omega}}(\mathbf{s'})_{\mathbf{a}^\star} - \hat{Q}_{\mathbf{\omega}}(\mathbf{s})_{\mathbf{a}} \right] ~ \nabla_{\mathbf{\omega}} \hat{Q}_{\mathbf{\omega}}(\mathbf{s})_{\mathbf{a}}$ <br>
		$\quad\quad$ $\mathbf{s} \leftarrow \mathbf{s'}$ <br>
	$\quad$ <b>UNTIL</b> $\mathbf{s}$ is final<br><br>
<b>RETURN</b> $\mathbf{\omega}$ <br>

#### Implement the Q-network

The Q-Network is used to approximate the action value function, which gives the expected future reward for taking a particular action in a particular state. The network is trained to minimize the difference between its predicted Q-values and the actual return received.

**Task 2.1:** implement the constructor and the `forward` method of the Q-network we will use in our RL agents

In [ ]:
class QNetwork(torch.nn.Module):
    """
    A Q-Network implemented with PyTorch.

    Attributes
    ----------
    layer1 : torch.nn.Linear
        First fully connected layer.
    layer2 : torch.nn.Linear
        Second fully connected layer.
    layer3 : torch.nn.Linear
        Third fully connected layer.

    Methods
    -------
    forward(x: torch.Tensor) -> torch.Tensor
        Define the forward pass of the QNetwork.
    """

    def __init__(self, n_observations: int, n_actions: int, nn_l1: int, nn_l2: int):
        """
        Initialize a new instance of QNetwork.

        Parameters
        ----------
        n_observations : int
            The size of the observation space.
        n_actions : int
            The size of the action space.
        nn_l1 : int
            The number of neurons on the first layer.
        nn_l2 : int
            The number of neurons on the second layer.
        """
        super(QNetwork, self).__init__()

        self.layer1 = torch.nn.Linear(n_observations, nn_l1)
        self.layer2 = torch.nn.Linear(nn_l1, nn_l2)
        self.layer3 = torch.nn.Linear(nn_l2, n_actions)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Define the forward pass of the QNetwork.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor (state).

        Returns
        -------
        torch.Tensor
            The output tensor (Q-values).
        """

        ### BEGIN SOLUTION ###

        # [...]
        x = ...

        ### END SOLUTION ###

        return x

#### Implement an inference function

**Task 2.2:** Your next assignment is to complete the function below, which will be used to evaluate the performance of an agent in a simulated environment over one or multiple episodes.

In [ ]:
def test_q_network_agent(env: gym.Env, q_network: torch.nn.Module, num_episode: int = 1, render: bool = True) -> List[int]:
    """
    Test a naive agent in the given environment using the provided Q-network.

    Parameters
    ----------
    env : gym.Env
        The environment in which to test the agent.
    q_network : torch.nn.Module
        The Q-network to use for decision making.
    num_episode : int, optional
        The number of episodes to run, by default 1.
    render : bool, optional
        Whether to render the environment, by default True.

    Returns
    -------
    List[int]
        A list of rewards per episode.
    """
    episode_reward_list = []

    for episode_id in range(num_episode):

        state, info = env.reset()
        done = False
        episode_reward = 0

        while not done:
            if render:
                env.render_wrapper.render()

            # Convert the state to a PyTorch tensor and add a batch dimension (unsqueeze)
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

            ### BEGIN SOLUTION ###

            q_values = ...
            action = ...

            ### END SOLUTION ###

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            episode_reward += reward

            state = next_state

        episode_reward_list.append(episode_reward)
        print(f"Episode reward: {episode_reward}")

    return episode_reward_list

**Task 2.3:** Test this function on the untrained agent.

In [ ]:
q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=5)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab6_dqn_naive_untained")

#### Implement the epsilon greedy function

**Task 2.4:** Now, let's proceed to implement the epsilon-greedy strategy, which is a crucial component in balancing exploration and exploitation during the learning process of our reinforcement learning agent. To accomplish this, complete the `__call__` function in the following code block.

In [ ]:
class EpsilonGreedy:
    """
    An Epsilon-Greedy policy.

    Attributes
    ----------
    epsilon : float
        The initial probability of choosing a random action.
    epsilon_min : float
        The minimum probability of choosing a random action.
    epsilon_decay : float
        The decay rate for the epsilon value after each action.
    env : gym.Env
        The environment in which the agent is acting.
    q_network : torch.nn.Module
        The Q-Network used to estimate action values.

    Methods
    -------
    __call__(state: np.ndarray) -> np.int64
        Select an action for the given state using the epsilon-greedy policy.
    decay_epsilon()
        Decay the epsilon value after each action.
    """

    def __init__(self,
                 epsilon_start: float,
                 epsilon_min: float,
                 epsilon_decay:float,
                 env: gym.Env,
                 q_network: torch.nn.Module):
        """
        Initialize a new instance of EpsilonGreedy.

        Parameters
        ----------
        epsilon_start : float
            The initial probability of choosing a random action.
        epsilon_min : float
            The minimum probability of choosing a random action.
        epsilon_decay : float
            The decay rate for the epsilon value after each episode.
        env : gym.Env
            The environment in which the agent is acting.
        q_network : torch.nn.Module
            The Q-Network used to estimate action values.
        """
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.env = env
        self.q_network = q_network

    def __call__(self, state: np.ndarray) -> np.int64:
        """
        Select an action for the given state using the epsilon-greedy policy.

        If a randomly chosen number is less than epsilon, a random action is chosen.
        Otherwise, the action with the highest estimated action value is chosen.

        Parameters
        ----------
        state : np.ndarray
            The current state of the environment.

        Returns
        -------
        np.int64
            The chosen action.
        """

        if random.random() < self.epsilon:
            action = self.env.action_space.sample()
        else:
            with torch.no_grad():

                ### BEGIN SOLUTION ###

                q_values = ...
                action = ...

                ### END SOLUTION ###

        return action

    def decay_epsilon(self):
        """
        Decay the epsilon value after each episode.

        The new epsilon value is the maximum of `epsilon_min` and the product of the current 
        epsilon value and `epsilon_decay`.
        """
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

#### Implementing a Learning Rate Scheduler

The following cell introduces a PyTorch Learning Rate (LR) scheduler. This scheduler is used for managing and adjusting the learning rate throughout the training process of our agent. It's designed to adjust the learning rate of an optimizer at each epoch, following an exponential decay strategy, but with a lower limit on the learning rate.

In [ ]:
class MinimumExponentialLR(torch.optim.lr_scheduler.ExponentialLR):
    def __init__(self, optimizer: torch.optim.Optimizer, lr_decay: float, last_epoch: int = -1, min_lr: float = 1e-6):
        """
        Initialize a new instance of MinimumExponentialLR.

        Parameters
        ----------
        optimizer : torch.optim.Optimizer
            The optimizer whose learning rate should be scheduled.
        lr_decay : float
            The multiplicative factor of learning rate decay.
        last_epoch : int, optional
            The index of the last epoch. Default is -1.
        min_lr : float, optional
            The minimum learning rate. Default is 1e-6.
        """
        self.min_lr = min_lr
        super().__init__(optimizer, lr_decay, last_epoch=-1)

    def get_lr(self) -> List[float]:
        """
        Compute learning rate using chainable form of the scheduler.

        Returns
        -------
        List[float]
            The learning rates of each parameter group.
        """
        return [
            max(base_lr * self.gamma ** self.last_epoch, self.min_lr)
            for base_lr in self.base_lrs
        ]

#### Implementing the Training Function

The following function is the final component of our initial agent. It orchestrates the training process, enabling the agent to learn from its interactions with the environment.

During each episode, the agent selects actions based on an epsilon-greedy policy, observes the next state and reward from the environment, and updates the weights of the Q-Network based on the observed reward and the maximum predicted Q-value of the next state.

**Task 2.5:** complete this function

In [ ]:
def train_naive_agent(env: gym.Env,
                      q_network: torch.nn.Module,
                      optimizer: torch.optim.Optimizer,
                      loss_fn: Callable,
                      epsilon_greedy: EpsilonGreedy,
                      device: torch.device,
                      lr_scheduler: _LRScheduler,
                      num_episodes: int,
                      gamma: float) -> List[float]:
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():


            # Get action, next_state and reward

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            episode_reward += reward

            # Update the q_network weights

            ### BEGIN SOLUTION ###

            target = ...
            q_value_of_current_action = ...

            ### END SOLUTION ###

            loss = loss_fn(q_value_of_current_action, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            lr_scheduler.step()

            if done:
                break

            state = next_state


        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

#### Train the agent

In [ ]:
env = gym.make('CartPole-v1')

NUMBER_OF_TRAININGS = DEFAULT_NUMBER_OF_TRAININGS    # Change the default (global) value here if you want a specific number of trainings for this exercise
naive_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # Instantiate required objects

    q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)
    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()

    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)

    # Train the q-network

    episode_reward_list = train_naive_agent(env,
                                            q_network,
                                            optimizer,
                                            loss_fn,
                                            epsilon_greedy,
                                            device,
                                            lr_scheduler,
                                            num_episodes=150,
                                            gamma=0.9)
    naive_trains_result_list[0].extend(range(len(episode_reward_list)))
    naive_trains_result_list[1].extend(episode_reward_list)
    naive_trains_result_list[2].extend([train_index for _ in episode_reward_list])

naive_trains_result_df = pd.DataFrame(np.array(naive_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
naive_trains_result_df["agent"] = "Naive"

# Save the action-value estimation function of the last train

torch.save(q_network, MODELS_DIR / "naive_q_network.pth")

env.close()

#### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=naive_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", hue="agent", kind="line", data=naive_trains_result_df, height=7, aspect=2)

#### Test it

In [ ]:
# q_network = torch.load("naive_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=5)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab6_dqn_naive_tained")

#### Why It Doesn't Work: The Complexity of Deep Reinforcement Learning

Our initial deep value-based agent did not converge, primarily due to the three fundamental challenges of value-based deep reinforcement learning:

1. **Coverage**: Convergence to the optimal Q-function relies on comprehensive coverage of the state space. However, in the context of deep RL, the state space is often too large to be fully covered. In situations where not all states are sampled due to their vast number, the guarantee of convergence no longer holds.

2. **Correlation**: The probability of transitioning to the next state is highly influenced by the current state. This strong correlation can lead to local overfitting and the risk of becoming trapped in a local optimum: the neural network, which approximates the Q-function, may become overly specialized in a small portion of the action-state space and neglect the rest.

3. **Convergence**: The "targets" used as the truth to be achieved "move" during the learning process. For the same prediction (estimation of the value of a state-action pair, i.e., its Q-value), the loss of a given example changes during the learning process (due to *bootstrapping* a main concept of TD-Learning). In other words, DQN tries to minimize a moving target, a target that depends on the model we are learning and optimizing. This can lead to instability and make it difficult for the learning process to converge to an optimal policy.

In the following sections, we will explore strategies to address these challenges and improve the performance of our deep reinforcement learning agent.

### Exercise 3: Implement Deep Q-Networks v1 (DQN version 2013 with experience replay)

In 2013, DeepMind made a significant contribution to the field of reinforcement learning with the publication of the paper "Playing Atari with Deep Reinforcement Learning" by Volodymyr Mnih and al (https://arxiv.org/abs/1312.5602). This paper marked the introduction of the first version of Deep Q-Networks (DQN).

The paper's primary innovation was the development of a technique to decorrelate states in reinforcement learning. This technique, known as *experience replay*, leverages a *replay buffer* to store and sample experiences. The introduction of experience replay greatly enhanced the stability and efficiency of the learning process.

#### Experience replay

Experience replay is a key technique used in Deep Q-Networks (DQN) to address the issues of correlation.

In a typical reinforcement learning setup, an agent learns by interacting with the environment, receiving feedback in the form of rewards, and updating its policy based on this feedback. This process is inherently sequential and the successive states are highly correlated, which can lead to overfitting and instability in learning.

Experience replay addresses these issues by storing the agent's experiences, i.e., the tuples of (state, action, reward, next state), in a data structure known as a replay buffer. During the learning process, instead of learning from the most recent experience, the agent randomly samples a batch of experiences from the replay buffer. This random sampling breaks the correlation between successive experiences, leading to more stable and robust learning.

Also, by learning from past experiences, the agent can effectively learn from a fixed target, which mitigates the issue of learning from a moving target. This is because the experiences in the replay buffer remain fixed once they are stored, even though the agent's policy continues to evolve.

#### DQN v2013 Algorithm

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br>
	$\quad\quad$ capacity of the experience replay memory $M$<br>
	$\quad\quad$ batch size $m$<br><br>

<b>Initialize</b> replay memory $\mathcal{D}$ to capacity $M$<br>
<b>Initialize</b> action-value function $\hat{Q}$ with random weights $\mathbf{\omega}$<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\mathbf{s} \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $\mathbf{a} \leftarrow \epsilon\text{-greedy}(\mathbf{s}, \hat{Q}_{\mathbf{\omega}}$ $)$<br>
		$\quad\quad$ $r, \mathbf{s'} \leftarrow \text{env.step}(\mathbf{a})$<br>
		$\quad\quad$ Store transition $(\mathbf{s}, \mathbf{a}, r, \mathbf{s'})$ in $\mathcal{D}$<br>
		$\quad\quad$ Sample random batch of transitions $(\mathbf{s}_j, \mathbf{a}_j, r_j, \mathbf{s'}_j)$ from $\mathcal{D}$<br>
		$\quad\quad$ Set $y_j = 
		\begin{cases} 
		r_j & \text{for terminal } \mathbf{s'}_j\\
		r_j + \gamma \max_{\mathbf{a}^\star}\hat{Q}_{\mathbf{\omega}}(\mathbf{s'}_j)_{\mathbf{a}^\star} & \text{for non-terminal } \mathbf{s'}_j
		\end{cases}$<br>
		$\quad\quad$ Perform a gradient descent step on $\left( y_j - \hat{Q}_{\mathbf{\omega}}(\mathbf{s}_j)_{\mathbf{a}_j} \right)^2$ with respect to the weights $\mathbf{\omega}$<br>
		$\quad\quad$ $\mathbf{s} \leftarrow \mathbf{s'}$ <br>
	$\quad$ <b>UNTIL</b> $\mathbf{s}$ is final<br><br>
<b>RETURN</b> $\mathbf{\omega}$ <br>


#### Implement the Replay Buffer

To incorporate experience replay into the provided naive deep value-based reinforcement learning agent definition, we need to introduce a memory buffer where experiences are stored, and then update the algorithm to sample a random batch of experiences from this buffer to update the weights.

In [ ]:
class ReplayBuffer:
    """
    A Replay Buffer.

    Attributes
    ----------
    buffer : collections.deque
        A double-ended queue where the transitions are stored.

    Methods
    -------
    add(state: np.ndarray, action: np.int64, reward: float, next_state: np.ndarray, done: bool)
        Add a new transition to the buffer.
    sample(batch_size: int) -> Tuple[np.ndarray, float, float, np.ndarray, bool]
        Sample a batch of transitions from the buffer.
    __len__()
        Return the current size of the buffer.
    """

    def __init__(self, capacity: int):
        """
        Initializes a ReplayBuffer instance.

        Parameters
        ----------
        capacity : int
            The maximum number of transitions that can be stored in the buffer.
        """
        self.buffer = collections.deque(maxlen=capacity)

    def add(self, state: np.ndarray, action: np.int64, reward: float, next_state: np.ndarray, done: bool):
        """
        Add a new transition to the buffer.

        Parameters
        ----------
        state : np.ndarray
            The state vector of the added transition.
        action : np.int64
            The action of the added transition.
        reward : float
            The reward of the added transition.
        next_state : np.ndarray
            The next state vector of the added transition.
        done : bool
            The final state of the added transition.
        """
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size: int) -> Tuple[np.ndarray, float, float, np.ndarray, bool]:
        """
        Sample a batch of transitions from the buffer.

        Parameters
        ----------
        batch_size : int
            The number of transitions to sample.

        Returns
        -------
        Tuple[np.ndarray, float, float, np.ndarray, bool]
            A batch of `batch_size` transitions.
        """
        # Here, `random.sample(self.buffer, batch_size)`
        # returns a list of tuples `(state, action, reward, next_state, done)`
        # where:
        # - `state`  and `next_state` are numpy arrays
        # - `action` and `reward` are floats
        # - `done` is a boolean
        #
        # `states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))`
        # generates 5 tuples `state`, `action`, `reward`, `next_state` and `done`, each having `batch_size` elements.
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return np.array(states), actions, rewards, np.array(next_states), dones

    def __len__(self):
        """
        Return the current size of the buffer.

        Returns
        -------
        int
            The current size of the buffer.
        """
        return len(self.buffer)

#### Implement the training function

The training function of our initial deep value-based agent needs to be modified to incorporate the use of the replay buffer effectively.

1. **Store Experiences**: After the agent takes an action and receives a reward and the next state from the environment, store this experience in the replay buffer.

2. **Sample Experiences**: Instead of using the most recent experience to update the agent's policy, randomly sample a batch of experiences from the replay buffer.

3. **Compute Loss and Update Weights**: Use the sampled experiences to compute the loss and update the weights of the Q-Network.

4. **Handle Terminal States**: If the 'done' flag of an experience is True, indicating a terminal state, make sure to adjust the target Q-value to be just the received reward. This is because there are no future rewards possible after a terminal state.

**Task 3.1:** complete the `train_dqn1_agent` to use the replay buffer.

In [ ]:
def train_dqn1_agent(env: gym.Env,
                     q_network: torch.nn.Module,
                     optimizer: torch.optim.Optimizer,
                     loss_fn: Callable,
                     epsilon_greedy: EpsilonGreedy,
                     device: torch.device,
                     lr_scheduler: _LRScheduler,
                     num_episodes: int,
                     gamma: float,
                     batch_size: int,
                     replay_buffer: ReplayBuffer) -> List[float]:
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.
    batch_size : int
        The size of the batch to use for training.
    replay_buffer : ReplayBuffer
        The replay buffer storing the experiences with their priorities.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():

            # Get action, next_state and reward

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            replay_buffer.add(state, action, reward, next_state, done)

            episode_reward += reward

            # Update the q_network weights with a batch of experiences from the buffer

            if len(replay_buffer) > batch_size:
                batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = replay_buffer.sample(batch_size)

                # Convert to PyTorch tensors
                batch_states_tensor = torch.tensor(batch_states, dtype=torch.float32, device=device)
                batch_actions_tensor = torch.tensor(batch_actions, dtype=torch.long, device=device)
                batch_rewards_tensor = torch.tensor(batch_rewards, dtype=torch.float32, device=device)
                batch_next_states_tensor = torch.tensor(batch_next_states, dtype=torch.float32, device=device)
                batch_dones_tensor = torch.tensor(batch_dones, dtype=torch.float32, device=device)

                ### BEGIN SOLUTION ###

                targets = ...
                current_q_values = ...

                ### END SOLUTION ###

                # Compute loss
                loss = loss_fn(current_q_values, targets)

                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                lr_scheduler.step()

            if done:
                break

            state = next_state

        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

#### Train it

In [ ]:
env = gym.make('CartPole-v1')

NUMBER_OF_TRAININGS = DEFAULT_NUMBER_OF_TRAININGS    # Change the default (global) value here if you want a specific number of trainings for this exercise
dqn1_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # Instantiate required objects
    
    q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)
    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()
    
    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)
    
    replay_buffer = ReplayBuffer(2000)
    
    # Train the q-network

    episode_reward_list = train_dqn1_agent(env,
                                           q_network,
                                           optimizer,
                                           loss_fn,
                                           epsilon_greedy,
                                           device,
                                           lr_scheduler,
                                           num_episodes=150,
                                           gamma=0.9,
                                           batch_size=128,
                                           replay_buffer=replay_buffer)
    dqn1_trains_result_list[0].extend(range(len(episode_reward_list)))
    dqn1_trains_result_list[1].extend(episode_reward_list)
    dqn1_trains_result_list[2].extend([train_index for _ in episode_reward_list])

dqn1_trains_result_df = pd.DataFrame(np.array(dqn1_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
dqn1_trains_result_df["agent"] = "DQN 2013"

# Save the action-value estimation function

torch.save(q_network, MODELS_DIR / "dqn1_q_network.pth")

env.close()

#### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=dqn1_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
#g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=dqn1_trains_result_df, height=7, aspect=2)

In [ ]:
all_trains_result_df = pd.concat([naive_trains_result_df, dqn1_trains_result_df])
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=all_trains_result_df, height=7, aspect=2)

#### Test it

In [ ]:
# q_network = torch.load("dqn1_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=3)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab6_dqn1_tained")

#### Score

In [ ]:
score_ex3 = dqn1_trains_result_df[["num_episodes", "mean_final_episode_reward"]].groupby("num_episodes").mean().max()
score_ex3

### Exercise 4: Implement Deep Q-Networks v2 (DQN version 2015) with *infrequent weight updates*

In 2015, DeepMind further advanced the field of reinforcement learning with the publication of the paper "Human-level control through deep reinforcement learning" by Volodymyr Mnih and colleagues (https://www.nature.com/articles/nature14236). This work introduced the second version of Deep Q-Networks (DQN).

<img src="https://github.com/jeremiedecock/polytechnique-inf639-2024-students/blob/main/assets/lab1_dqn_nature_journal.jpg?raw=true" width="200px" />

The key contribution of this paper was the introduction of a method to stabilize the learning process by infrequently updating the target weights. This technique, known as *infrequent updates of target weights*, significantly improved the stability of the learning process.

#### Infrequent weight updates

Infrequent weight updates, also known as the use of a target network, is a technique used in Deep Q-Networks (DQN) to address the issue of learning from a moving target.

In a typical DQN setup, there are two neural networks: the Q-network and the target network. The Q-network is used to predict the Q-values and is updated at every time step. The target network is used to compute the target Q-values for the update, and its weights are updated less frequently, typically every few thousand steps, by copying the weights from the Q-network.

The idea behind infrequent weight updates is to stabilize the learning process by keeping the target Q-values fixed for a number of steps. This mitigates the issue of learning from a moving target, as the target Q-values remain fixed between updates.

Without infrequent weight updates, both the predicted and target Q-values would change at every step, which could lead to oscillations and divergence in the learning process. By introducing a delay between updates of the target Q-values, the risk of such oscillations is reduced.

#### DQN v2015 Algorithm

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in (0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br>
	$\quad\quad$ capacity of the experience replay memory $M$<br>
	$\quad\quad$ batch size $m$<br>
	$\quad\quad$ target network update frequency $\tau$<br><br>

<b>Initialize</b> replay memory $\mathcal{D}$ to capacity $M$<br>
<b>Initialize</b> action-value function $\hat{Q}$ with random weights $\mathbf{\omega}$<br>
<b>Initialize</b> target action-value function $\hat{Q}$ with weights $\mathbf{\omega_2} = \mathbf{\omega}$<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\mathbf{s} \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $\mathbf{a} \leftarrow \epsilon\text{-greedy}(\mathbf{s}, \hat{Q}_{\mathbf{\omega_1}}$ $)$<br>
		$\quad\quad$ $r, \mathbf{s'} \leftarrow \text{env.step}(\mathbf{a})$<br>
		$\quad\quad$ Store transition $(\mathbf{s}, \mathbf{a}, r, \mathbf{s'})$ in $\mathcal{D}$<br>
		$\quad\quad$ Sample random batch of transitions $(\mathbf{s}_j, \mathbf{a}_j, r_j, \mathbf{s'}_j)$ from $\mathcal{D}$<br>
		$\quad\quad$ For each $j$, set $y_j = 
		\begin{cases} 
		r_j & \text{for terminal } \mathbf{s'}_j\\
		r_j + \gamma \max_{\mathbf{a}^\star} \hat{Q}_{\mathbf{\omega_2}} (\mathbf{s'}_j)_{\mathbf{a}^\star} & \text{for non-terminal } \mathbf{s'}_j
		\end{cases}$<br>
		$\quad\quad$ Perform a gradient descent step on $\left( y_j - \hat{Q}_{\mathbf{\omega_1}}(\mathbf{s}_j)_{\mathbf{a}_j} \right)^2$ with respect to the weights $\mathbf{\omega_1}$<br>
		$\quad\quad$ Every $\tau$ steps reset $\hat{Q}_{\mathbf{\omega_2}}$ to $\hat{Q}_{\mathbf{\omega_1}}$, i.e., set $\mathbf{\omega_2} \leftarrow \mathbf{\omega_1}$<br>
		$\quad\quad$ $\mathbf{s} \leftarrow \mathbf{s'}$ <br>
	$\quad$ <b>UNTIL</b> $\mathbf{s}$ is final<br><br>
<b>RETURN</b> $\mathbf{\omega}$ <br>


#### Implement the training function

To incorporate the use of infrequent weight updates in the training function, you would need to make the following modifications:

1. **Update the Target Network Infrequently**: Instead of updating the weights of the target network at every time step, update them less frequently, for example, every few thousand steps. The weights of the target network are updated by copying the weights from the Q-network.

2. **Compute Target Q-values with the Target Network**: When computing the target Q-values for the update, use the target network instead of the Q-network. This ensures that the target Q-values remain fixed between updates, which stabilizes the learning process.

**Task 4.1:** complete the `train_dqn2_agent` to apply infrequent weight updates.

In [ ]:
def train_dqn2_agent(env: gym.Env,
                     q_network: torch.nn.Module,
                     target_q_network: torch.nn.Module,
                     optimizer: torch.optim.Optimizer,
                     loss_fn: Callable,
                     epsilon_greedy: EpsilonGreedy,
                     device: torch.device,
                     lr_scheduler: _LRScheduler,
                     num_episodes: int,
                     gamma: float,
                     batch_size: int,
                     replay_buffer: ReplayBuffer,
                     target_q_network_sync_period: int) -> List[float]:
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    target_q_network : torch.nn.Module
        The target Q-network to use for estimating the target Q-values.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.
    batch_size : int
        The size of the batch to use for training.
    replay_buffer : ReplayBuffer
        The replay buffer storing the experiences with their priorities.
    target_q_network_sync_period : int
        The number of episodes after which the target Q-network should be updated with the weights of the Q-network.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    iteration = 0
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():

            # Get action, next_state and reward

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            replay_buffer.add(state, action, reward, next_state, done)

            episode_reward += reward

            # Update the q_network weights with a batch of experiences from the buffer

            if len(replay_buffer) > batch_size:
                batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = replay_buffer.sample(batch_size)

                # Convert to PyTorch tensors
                batch_states_tensor = torch.tensor(batch_states, dtype=torch.float32, device=device)
                batch_actions_tensor = torch.tensor(batch_actions, dtype=torch.long, device=device)
                batch_rewards_tensor = torch.tensor(batch_rewards, dtype=torch.float32, device=device)
                batch_next_states_tensor = torch.tensor(batch_next_states, dtype=torch.float32, device=device)
                batch_dones_tensor = torch.tensor(batch_dones, dtype=torch.float32, device=device)

                ### BEGIN SOLUTION ###

                targets = ...
                current_q_values = ...

                ### END SOLUTION ###

                # Compute loss
                loss = loss_fn(current_q_values, targets)

                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                lr_scheduler.step()

            # Update the target q-network

            ### BEGIN SOLUTION ###

            # Every few training steps (e.g., every 100 steps), the weights of the target network are updated with the weights of the Q-network

            ### END SOLUTION ###

            iteration += 1

            if done:
                break

            state = next_state

        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

#### Train it

In order to test this new implementation, we needs de adapt the following cell to instantiate and initialize the two neural networks.

**Task 4.2:** complete the following cell to make the two Q-Networks. Initialize a target network that has the same architecture as the Q-network. The weights of the target network are initially copied from the Q-network.

In [ ]:
env = gym.make('CartPole-v1')

# NUMBER_OF_TRAININGS = 20
dqn2_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # Instantiate required objects

    ### BEGIN SOLUTION ###

    q_network = ...
    target_q_network = ...
    # Initialize the target Q-network with the same weights as the Q-network

    ### END SOLUTION ###

    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()

    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)

    replay_buffer = ReplayBuffer(2000)

    # Train the q-network

    episode_reward_list = train_dqn2_agent(env,
                                           q_network,
                                           target_q_network,
                                           optimizer,
                                           loss_fn,
                                           epsilon_greedy,
                                           device,
                                           lr_scheduler,
                                           num_episodes=150,
                                           gamma=0.9,
                                           batch_size=128,
                                           replay_buffer=replay_buffer,
                                           target_q_network_sync_period=30)
    dqn2_trains_result_list[0].extend(range(len(episode_reward_list)))
    dqn2_trains_result_list[1].extend(episode_reward_list)
    dqn2_trains_result_list[2].extend([train_index for _ in episode_reward_list])

dqn2_trains_result_df = pd.DataFrame(np.array(dqn2_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
dqn2_trains_result_df["agent"] = "DQN 2015"

# Save the action-value estimation function

torch.save(q_network, MODELS_DIR / "dqn2_q_network.pth")

env.close()

#### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=dqn2_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
#g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=dqn2_trains_result_df, height=7, aspect=2)

In [ ]:
all_trains_result_df = pd.concat([naive_trains_result_df, dqn1_trains_result_df, dqn2_trains_result_df])
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=all_trains_result_df, height=7, aspect=2)

#### Test it

In [ ]:
# q_network = torch.load("dqn2_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=3)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab6_dqn2_tained")

#### Score

In [ ]:
score_ex4 = dqn2_trains_result_df[["num_episodes", "mean_final_episode_reward"]].groupby("num_episodes").mean().max()
score_ex4

## Part 3: Double Deep Q-Network (DDQN)

Hado Van Hasselt et al. introduced Double Deep Q-Networks in the publication "Deep reinforcement learning with Double Q-Learning" in 2016 (https://arxiv.org/abs/1509.06461).

Double Deep Q-Networks (DDQN) is an enhancement over the standard Deep Q-Network (DQN). It was designed to reduce the overestimation of action values that can occur in DQN. The fundamental concept behind DDQN is the separation of action selection from their evaluation to reduce the overestimation of action values in DQN.

### DDQN Algorithm

<b>Input</b>:<br>
    $\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
    $\quad\quad$ discount factor $\gamma$<br>
    $\quad\quad$ step size $\alpha \in (0,1]$<br>
    $\quad\quad$ small $\epsilon > 0$<br>
    $\quad\quad$ capacity of the experience replay memory $M$<br>
    $\quad\quad$ batch size $m$<br>
    $\quad\quad$ target network update frequency $\tau$<br><br>

<b>Initialize</b> replay memory $\mathcal{D}$ to capacity $M$<br>
<b>Initialize</b> action-value function $\hat{Q}$ with random weights $\mathbf{\omega_1}$<br>
<b>Initialize</b> target action-value function $\hat{Q}$ with weights $\mathbf{\omega_2} = \mathbf{\omega_1}$<br><br>

<b>FOR EACH</b> episode<br>
    $\quad$ $\mathbf{s} \leftarrow \text{env.reset}()$<br>
    $\quad$ <b>DO</b> <br>
        $\quad\quad$ $\mathbf{a} \leftarrow \epsilon\text{-greedy}(\mathbf{s}, \hat{Q}_{\mathbf{\omega_1}})$<br>
        $\quad\quad$ $r, \mathbf{s'} \leftarrow \text{env.step}(\mathbf{a})$<br>
        $\quad\quad$ Store transition $(\mathbf{s}, \mathbf{a}, r, \mathbf{s'})$ in $\mathcal{D}$<br>
        $\quad\quad$ Sample random batch of transitions $(\mathbf{s}_j, \mathbf{a}_j, r_j, \mathbf{s'}_j)$ from $\mathcal{D}$<br>
        $\quad\quad$ For each $j$, set $\textcolor{red}{\mathbf{a}^\star = \arg\max_{\mathbf{a}} \hat{Q}_{\mathbf{\omega_1}}(\mathbf{s'}_j)_{\mathbf{a}}}$<br>
        $\quad\quad$ Set $y_j = 
        \begin{cases} 
        r_j & \text{for terminal } \mathbf{s'}_j\\
        r_j + \gamma \hat{Q}_{\mathbf{\omega_2}}(\mathbf{s'}_j)_{\textcolor{red}{\mathbf{a}^\star}} & \text{for non-terminal } \mathbf{s'}_j
        \end{cases}$<br>
        $\quad\quad$ Perform a gradient descent step on $\left( y_j - \hat{Q}_{\mathbf{\omega_1}}(\mathbf{s}_j)_{\mathbf{a}_j} \right)^2$ with respect to the weights $\mathbf{\omega_1}$<br>
        $\quad\quad$ Every $\tau$ steps reset $\hat{Q}_{\mathbf{\omega_2}}$ to $\hat{Q}_{\mathbf{\omega_1}}$, i.e., set $\mathbf{\omega_2} \leftarrow \mathbf{\omega_1}$<br>
        $\quad\quad$ $\mathbf{s} \leftarrow \mathbf{s'}$ <br>
    $\quad$ <b>UNTIL</b> $\mathbf{s}$ is final<br><br>
<b>RETURN</b> $\mathbf{\omega_1}$ <br>


### Exercise 5: Implement DDQN

#### Implement the training function

Switching from a Deep Q-Network (DQN) to a Double Deep Q-Network (DDQN) involves a key modification in the way the Q-value update is performed during training. 

In DQN, the Q-value update is done using the maximum Q-value for the next state from the target network. However, this can lead to an overestimation of Q-values because it always uses the maximum estimate.

DDQN addresses this by decoupling the selection of the action from the evaluation of that action. In DDQN, the Q-network is used to select what the next action is, and the target network is used to evaluate the Q-value of taking that action at the next state.

**Task 4.2:** complete the `train_ddqn_agent` function.

In [ ]:
def train_ddqn_agent(env: gym.Env,
                     q_network: torch.nn.Module,
                     target_q_network: torch.nn.Module,
                     optimizer: torch.optim.Optimizer,
                     loss_fn: Callable,
                     epsilon_greedy: EpsilonGreedy,
                     device: torch.device,
                     lr_scheduler: _LRScheduler,
                     num_episodes: int,
                     gamma: float,
                     batch_size: int,
                     replay_buffer: ReplayBuffer,
                     target_q_network_sync_period: int) -> List[float]:
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    target_q_network : torch.nn.Module
        The target Q-network to use for estimating the target Q-values.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.
    batch_size : int
        The size of the batch to use for training.
    replay_buffer : ReplayBuffer
        The replay buffer storing the experiences with their priorities.
    target_q_network_sync_period : int
        The number of episodes after which the target Q-network should be updated with the weights of the Q-network.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    iteration = 0
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():

            # GET ACTION, NEXT_STATE AND REWARD ###########

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            replay_buffer.add(state, action, reward, next_state, done)

            episode_reward += reward

            # UPDATE THE Q_NETWORK WEIGHTS WITH A BATCH OF EXPERIENCES FROM THE BUFFER

            if len(replay_buffer) > batch_size:
                batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = replay_buffer.sample(batch_size)

                # Convert to PyTorch tensors
                batch_states_tensor = torch.tensor(batch_states, dtype=torch.float32, device=device)
                batch_actions_tensor = torch.tensor(batch_actions, dtype=torch.long, device=device)
                batch_rewards_tensor = torch.tensor(batch_rewards, dtype=torch.float32, device=device)
                batch_next_states_tensor = torch.tensor(batch_next_states, dtype=torch.float32, device=device)
                batch_dones_tensor = torch.tensor(batch_dones, dtype=torch.float32, device=device)

                # Compute the target Q values for the batch
                with torch.no_grad():

                    ### BEGIN SOLUTION ###

                    targets = ...

                    ### END SOLUTION ###

                # Compute the current Q values for the batch.
                # 
                # The expression `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)` is used to select specific elements from the tensor of Q-values returned by the Q-network.
                # 
                # Here's a breakdown of the following line of code:
                # - `q_network(batch_states_tensor)`:
                #   This is passing a batch of states through the Q-network.
                #   For each state, this outputs the Q-value for each possible action.
                #   Thus, `q_network(batch_states_tensor)` returns a tensor of shape (batch_size, action_dim).
                # 
                # - `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1))`:
                #   This is selecting the Q-values corresponding to the actions that were actually taken.
                #   The `gather` function is used to select elements from a tensor using an index.
                #   In this case, the index is `batch_actions_tensor.unsqueeze(-1)`, which is a tensor of the actions that were taken.
                #   The `unsqueeze(-1)` function is used to add an extra dimension to the tensor, which is necessary for the `gather` function.
                # 
                # - `squeeze(-1)`:
                #   This is removing the extra dimension that was added by `unsqueeze(-1)`.
                #   The `squeeze` function is used to remove dimensions of size 1 from a tensor.
                #
                # So, the entire expression is selecting the Q-values of the actions that were actually taken from the tensor of all Q-values,
                # and returning a tensor of these selected Q-values.
                current_q_values = q_network(batch_states_tensor).gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)

                # Compute loss
                loss = loss_fn(current_q_values, targets)

                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                lr_scheduler.step()

            # UPDATE THE TARGET Q-NETWORK #################

            # Every few training steps (e.g., every 100 steps), the weights of the target network are updated with the weights of the Q-network

            if iteration % target_q_network_sync_period == 0:
                target_q_network.load_state_dict(q_network.state_dict())

            iteration += 1

            if done:
                break

            state = next_state

        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

### Train it

In [ ]:
env = gym.make('CartPole-v1')

NUMBER_OF_TRAININGS = DEFAULT_NUMBER_OF_TRAININGS    # Change the default (global) value here if you want a specific number of trainings for this exercice
ddqn_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # Instantiate required objects

    q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)
    target_q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device) # The target Q-network is used to compute the target Q-values for the loss function
    target_q_network.load_state_dict(q_network.state_dict()) # Initialize the target Q-network with the same weights as the Q-network

    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()

    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)

    replay_buffer = ReplayBuffer(2000)

    # Train the q-network

    episode_reward_list = train_ddqn_agent(env,
                                           q_network,
                                           target_q_network,
                                           optimizer,
                                           loss_fn,
                                           epsilon_greedy,
                                           device,
                                           lr_scheduler,
                                           num_episodes=150,
                                           gamma=0.9,
                                           batch_size=128,
                                           replay_buffer=replay_buffer,
                                           target_q_network_sync_period=30)
    ddqn_trains_result_list[0].extend(range(len(episode_reward_list)))
    ddqn_trains_result_list[1].extend(episode_reward_list)
    ddqn_trains_result_list[2].extend([train_index for _ in episode_reward_list])

ddqn_trains_result_df = pd.DataFrame(np.array(ddqn_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
ddqn_trains_result_df["agent"] = "DDQN"

# SAVE THE ACTION-VALUE ESTIMATION FUNCTION

torch.save(q_network, MODELS_DIR / "ddqn_q_network.pth")

env.close()

### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=ddqn_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
#g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=ddqn_trains_result_df, height=7, aspect=2)

In [ ]:
all_trains_result_df = pd.concat([naive_trains_result_df, dqn1_trains_result_df, dqn2_trains_result_df, ddqn_trains_result_df])
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=all_trains_result_df, height=7, aspect=2)

### Test it

In [ ]:
# q_network = torch.load("ddqn_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=3)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_ddqn_tained")

### Experimental results

**Task 5.2:** What do you observe? Why?

## Part 4: Prioritized Experience Replay (PEX or PER)

Prioritized Experience Replay (PER) is an enhancement to the traditional experience replay mechanism used in Deep Q-Networks (DQNs) and other reinforcement learning algorithms. In traditional experience replay, experiences (or transitions) are stored in a replay buffer, and mini-batches are sampled uniformly at random from this buffer to update the agent's Q-network. This means every experience has the same probability of being chosen, regardless of its significance to the learning process.

Prioritized Experience Replay (PER) is a technique that modifies the standard experience replay by more frequently replaying experiences that have a high expected learning progress, as measured by their temporal-difference (TD) error.

Prioritized Experience Replay was introduced by Tom Schaul et al. in the paper "Prioritised experience replay" in 2016 (https://arxiv.org/abs/1511.05952).

### DQN with PER algorithm

The DQN with Prioritized Experience Replay (PER) algorithm is detailed in page 5 of the paper "Prioritised experience replay" (https://arxiv.org/abs/1511.05952).

### Exercise 6: Implement DQN with PER

To switch from a DQN to a DQN with Prioritized Experience Replay (PER), you need to make a few modifications to the *DQN 2015* agent:

1. **Experience Replay Buffer**: Replace the standard experience replay buffer with a prioritized one. This buffer should store experiences with a priority value that is updated after each learning step. The priority value is usually the absolute TD error plus a small constant to avoid experiences having zero probability of being chosen.
2. **Sampling Method**: Change the sampling method from uniform to prioritized sampling. This means experiences with higher priority values have a higher probability of being chosen for learning.
3. **Loss Function**: Modify the loss function to include importance sampling weights. These weights compensate for the bias introduced by the non-uniform sampling. The weight of each sampled experience is the inverse of its probability of being chosen.



#### Implement the PrioritizedReplayBuffer

**Task 6.1:** implement the `PrioritizedReplayBuffer` class.

In [ ]:
class PrioritizedReplayBuffer:
    """
    Implements a Prioritized Experience Replay buffer as described in the paper
    "Prioritized Experience Replay" (https://arxiv.org/abs/1511.05952).

    Attributes
    ----------
    buffer : Deque[Tuple[float, float, float, float, float]]
        The replay buffer storing the experiences.
    priorities : Deque[float]
        The priorities of the experiences in the buffer.
    """

    def __init__(self, capacity: int):
        """
        Initialize the replay buffer.

        Parameters
        ----------
        capacity : int
            The maximum number of experiences the buffer can hold.
        """
        self.buffer = collections.deque(maxlen=capacity)
        self.priorities = collections.deque(maxlen=capacity)

    def add(self, state: float, action: float, reward: float, next_state: float, done: float):
        """
        Add an experience to the buffer.

        Parameters
        ----------
        state : float
            The current state.
        action : float
            The action taken.
        reward : float
            The reward received.
        next_state : float
            The next state.
        done : float
            Whether the episode has ended.
        """
        self.buffer.append((state, action, reward, next_state, done))
        self.priorities.append(max(self.priorities, default=1))

    def sample(self, batch_size: int, alpha: float = 0.6, beta: float = 0.4) -> Tuple[List[Tuple[float, float, float, float, float]], np.ndarray, np.ndarray]:
        """
        Sample a batch of experiences from the buffer.

        Parameters
        ----------
        batch_size : int
            The number of experiences to sample.
        alpha : float, optional
            The exponent that determines how much prioritization is used.
        beta : float, optional
            The exponent that determines how much importance sampling is used.

        Returns
        -------
        Tuple[List[Tuple[float, float, float, float, float]], np.ndarray, np.ndarray]
            The sampled experiences, the indices of the sampled experiences, and the importance sampling weights.
        """
        
        ### BEGIN SOLUTION ###

        # Calculate priorities
        priorities = ...

        # Sample experiences
        indices = ...
        experiences = ...

        # Calculate weights
        weights = ...

        ### END SOLUTION ###

        return experiences, indices, weights

    def update_priorities(self, indices: np.ndarray, errors: List[float], offset: float = 0.1):
        """
        Update the priorities of the sampled experiences.

        Parameters
        ----------
        indices : np.ndarray
            The indices of the sampled experiences.
        errors : List[float]
            The new TD errors for the sampled experiences.
        offset : float, optional
            A small constant to ensure the priorities are strictly positive.
        """
        for i, error in zip(indices, errors):
            self.priorities[i] = error + offset

    def __len__(self) -> int:
        """
        Return the current size of the buffer.

        Returns
        -------
        int
            The current size of the buffer.
        """
        return len(self.buffer)

#### Implement the training function

**Task 6.2:** complete the `train_dqn_per_agent` function.

In [ ]:
def train_dqn_per_agent(env: gym.Env,
                        q_network: torch.nn.Module,
                        target_q_network: torch.nn.Module,
                        optimizer: torch.optim.Optimizer,
                        loss_fn: Callable,
                        epsilon_greedy: EpsilonGreedy,
                        device: torch.device,
                        lr_scheduler: _LRScheduler,
                        num_episodes: int,
                        gamma: float,
                        batch_size: int,
                        replay_buffer: PrioritizedReplayBuffer,  # Use a Prioritized Replay Buffer
                        target_q_network_sync_period: int,
                        alpha: float = 0.6,  # Alpha parameter for PER
                        beta: float = 0.4):  # Beta parameter for PER
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    target_q_network : torch.nn.Module
        The target Q-network to use for estimating the target Q-values.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.
    batch_size : int
        The size of the batch to use for training.
    replay_buffer : PrioritizedReplayBuffer
        The replay buffer storing the experiences with their priorities.
    target_q_network_sync_period : int
        The number of episodes after which the target Q-network should be updated with the weights of the Q-network.
    alpha : float
        The exponent that determines how much prioritization is used when sampling from the replay buffer.
    beta : float
        The exponent that determines how much importance sampling is used to adjust the loss function.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    iteration = 0
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():

            # GET ACTION, NEXT_STATE AND REWARD ###########

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            replay_buffer.add(state, action, reward, next_state, done)

            episode_reward += reward

            # UPDATE THE Q_NETWORK WEIGHTS WITH A BATCH OF EXPERIENCES FROM THE BUFFER

            if len(replay_buffer) > batch_size:
                # Sample experiences and importance-sampling weights from the buffer

                experiences, indices, weights = replay_buffer.sample(batch_size, alpha, beta)
                batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = zip(*experiences)

                # Convert to PyTorch tensors
                batch_states_tensor = torch.tensor(batch_states, dtype=torch.float32, device=device)
                batch_actions_tensor = torch.tensor(batch_actions, dtype=torch.long, device=device)
                batch_rewards_tensor = torch.tensor(batch_rewards, dtype=torch.float32, device=device)
                batch_next_states_tensor = torch.tensor(batch_next_states, dtype=torch.float32, device=device)
                batch_dones_tensor = torch.tensor(batch_dones, dtype=torch.float32, device=device)
                weights_tensor = torch.tensor(weights, dtype=torch.float32, device=device)

                # Compute the target Q values for the batch
                with torch.no_grad():
                    # Here's a breakdown of the next line of code:
                    # - `q_network(batch_next_states_t)`:
                    #   This is passing a batch of "next states" through the Q-network.
                    #   This outputs the Q-value for each possible action, a tensor of shape (batch_size, action_dim).
                    #
                    #  - `.max(dim=1)`:
                    #   This is finding the maximum Q-value for each state in the batch.
                    #   The dim=1 argument specifies that the maximum should be taken over the action dimension.
                    #
                    # The max() function in PyTorch returns a tuple containing two tensors: the maximum values and the indices where these maximum values were found.
                    # In the next lines of code, we will just use the first tensor (the maximum values) and ignoring the second tensor (the indices).
                    next_state_q_values, best_action_index = target_q_network(batch_next_states_tensor).max(dim=1)

                    # The targets for the batch are the rewards plus the discounted maximum Q-values obtained from the next states.
                    # The expression `(1 - batch_dones_tensor)` is used to handle the end of episodes.
                    # The `batch_dones_tensor` indicates whether each state in the batch is a terminal state (i.e., the end of an episode).
                    # If a state is a terminal state, the corresponding value in `batch_dones_tensor` is 1, otherwise it's 0.
                    # The Q-value of a terminal state is defined to be 0. Therefore, when calculating the target Q-values,
                    # we don't want to include the Q-value of the next state if the current state is a terminal state.
                    # This is achieved by multiplying `next_state_q_values` by `(1 - batch_dones_tensor)`.
                    # If the state is a terminal state, this expression becomes 0 and the Q-value of the next state is effectively ignored.
                    # If the state is not a terminal state, this expression is 1 and the Q-value of the next state is included in the calculation.
                    targets = batch_rewards_tensor + gamma * next_state_q_values * (1 - batch_dones_tensor)

                # Compute the current Q values for the batch.
                # 
                # The expression `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)` is used to select specific elements from the tensor of Q-values returned by the Q-network.
                # 
                # Here's a breakdown of the following line of code:
                # - `q_network(batch_states_tensor)`:
                #   This is passing a batch of states through the Q-network.
                #   For each state, this outputs the Q-value for each possible action.
                #   Thus, `q_network(batch_states_tensor)` returns a tensor of shape (batch_size, action_dim).
                # 
                # - `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1))`:
                #   This is selecting the Q-values corresponding to the actions that were actually taken.
                #   The `gather` function is used to select elements from a tensor using an index.
                #   In this case, the index is `batch_actions_tensor.unsqueeze(-1)`, which is a tensor of the actions that were taken.
                #   The `unsqueeze(-1)` function is used to add an extra dimension to the tensor, which is necessary for the `gather` function.
                # 
                # - `squeeze(-1)`:
                #   This is removing the extra dimension that was added by `unsqueeze(-1)`.
                #   The `squeeze` function is used to remove dimensions of size 1 from a tensor.
                #
                # So, the entire expression is selecting the Q-values of the actions that were actually taken from the tensor of all Q-values,
                # and returning a tensor of these selected Q-values.
                current_q_values = q_network(batch_states_tensor).gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)

                # Compute loss with importance-sampling weights

                ### BEGIN SOLUTION ###

                loss = ...

                ### END SOLUTION ###

                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                lr_scheduler.step()


                ### BEGIN SOLUTION ###

                # Update priorities in the buffer

                ### END SOLUTION ###

            # UPDATE THE TARGET Q-NETWORK #################

            # Every few training steps (e.g., every 100 steps), the weights of the target network are updated with the weights of the Q-network

            if iteration % target_q_network_sync_period == 0:
                target_q_network.load_state_dict(q_network.state_dict())

            iteration += 1

            if done:
                break

            state = next_state

        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

### Train it

In [ ]:
env = gym.make('CartPole-v1')

NUMBER_OF_TRAININGS = DEFAULT_NUMBER_OF_TRAININGS    # Change the default (global) value here if you want a specific number of trainings for this exercice
dqn_per_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # INSTANTIATE REQUIRED OBJECTS

    q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)
    target_q_network = QNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device) # The target Q-network is used to compute the target Q-values for the loss function
    target_q_network.load_state_dict(q_network.state_dict()) # Initialize the target Q-network with the same weights as the Q-network

    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()

    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)

    replay_buffer = PrioritizedReplayBuffer(2000)

    # TRAIN THE Q-NETWORK

    episode_reward_list = train_dqn_per_agent(env,
                                              q_network,
                                              target_q_network,
                                              optimizer,
                                              loss_fn,
                                              epsilon_greedy,
                                              device,
                                              lr_scheduler,
                                              num_episodes=150,
                                              gamma=0.9,
                                              batch_size=128,
                                              replay_buffer=replay_buffer,
                                              target_q_network_sync_period=30,
                                              alpha=0.6,
                                              beta=0.4)
    dqn_per_trains_result_list[0].extend(range(len(episode_reward_list)))
    dqn_per_trains_result_list[1].extend(episode_reward_list)
    dqn_per_trains_result_list[2].extend([train_index for _ in episode_reward_list])

dqn_per_trains_result_df = pd.DataFrame(np.array(dqn_per_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
dqn_per_trains_result_df["agent"] = "DQN 2015 + PER"

# SAVE THE ACTION-VALUE ESTIMATION FUNCTION

torch.save(q_network, MODELS_DIR / "dqn_per_q_network.pth")

env.close()

### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=dqn_per_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
#g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=dqn_per_trains_result_df, height=7, aspect=2)

In [ ]:
all_trains_result_df = pd.concat([naive_trains_result_df, dqn1_trains_result_df, dqn2_trains_result_df, ddqn_trains_result_df, dqn_per_trains_result_df])
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=all_trains_result_df, height=7, aspect=2)

### Test it

In [ ]:
# q_network = torch.load("dqn_per_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=3)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_dqn_per_tained")

### Experimental results

**Task 6.2:** What do you observe? Why?

## Part 5: Dueling Double DQN : DDQN with Advantage function


Dueling Double DQN (Dueling DDQN) is an enhancement over the standard DQN that aims to improve the quality of the learned value function (https://arxiv.org/abs/1511.06581). The motivation for Dueling DDQN lies in its architecture, which separately estimates two components: the value of being in a particular state (V(s)), and the advantage of taking a particular action in that state (A(s, a)).

In standard DQN, a single stream of network layers estimates the Q-value directly. In contrast, Dueling DDQN has two streams to separately estimate the value and advantage functions, which are then combined to calculate the Q-value. This allows the network to more effectively learn which states are (or are not) valuable without having to learn the effect of each action for each state. This is particularly useful in environments where the value of the state does not vary much across actions.

The separation of the estimation process helps in stabilizing learning and often leads to better policy evaluation, especially in cases where the action choice does not have a large impact on what happens next—making Dueling DDQN a more robust and often more efficient learning algorithm compared to the standard DQN.

Thus, the Dueling DQN architecture introduces two separate streams within the neural network:

   - **Value Stream (`V(s)`):** Estimates the overall value of being in a state, regardless of the action taken.
   - **Advantage Stream (`A(s, a)`):** Estimates the relative advantage of each action in a given state.

These two streams share some common layers at the beginning and then branch out. They are combined at the end to produce the Q-values using the following formula:

$$Q(s, a) = V(s) + \left( A(s, a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a') \right)$$

where:
- $A(s, a)$ is the advantage of action $a$ in state $s$.
- $|\mathcal{A}|$ is the number of possible actions.

<img src="https://www.researchgate.net/publication/336715783/figure/fig2/AS:816675293253632@1571721971001/Double-Dueling-Deep-Q-Learning-Network-DDDQN.jpg"></img>

### Exercise 7: Implement DDDQN

Modify the above `QNetwork` class to implement a **Dueling Double Deep Q-Network (DDDQN)** by completing the `__init__` and `forward` methods of the new class `DuelingQNetwork` partially implemented below.

### Definition of the Dueling DQN network

#### Implement the Q-Network

**Task 7.1:** complete the `DuelingDQN` class.

- **In the `__init__` Method:**
  - Initialize the shared layers (`layer1` and `layer2`) as in the original `QNetwork`.
  - Define two separate streams after the shared layers:
    - **Value Stream:**
      - A fully connected layer (`self.value_layer`) that outputs a single value.
    - **Advantage Stream:**
      - A fully connected layer (`self.advantage_layer`) that outputs a value for each possible action.

- **In the `forward` Method:**
  - Pass the input through the shared layers with activation functions.
  - Pass the output of the shared layers into both the value and advantage streams.
  - Combine the outputs of the value and advantage streams to compute the final Q-values using the formula provided.

In [ ]:
class DuelingQNetwork(torch.nn.Module):
    """
    A Dueling Q-Network implemented with PyTorch.

    Attributes
    ----------
    layer1 : torch.nn.Linear
        First fully connected layer (common feature layer).
    layer2 : torch.nn.Linear
        Second fully connected layer (common feature layer).
    value_layer : torch.nn.Linear
        Linear layer for the value stream.
    advantage_layer : torch.nn.Linear
        Linear layer for the advantage stream.

    Methods
    -------
    forward(x: torch.Tensor) -> torch.Tensor
        Define the forward pass of the DuelingQNetwork.
    """

    def __init__(self, n_observations: int, n_actions: int, nn_l1: int, nn_l2: int):
        """
        Initialize a new instance of DuelingQNetwork.

        Parameters
        ----------
        n_observations : int
            The size of the observation space.
        n_actions : int
            The size of the action space.
        nn_l1 : int
            The number of neurons in the first hidden layer.
        nn_l2 : int
            The number of neurons in the second hidden layer.
        """
        super(DuelingQNetwork, self).__init__()

        ### BEGIN SOLUTION ###

        ...

        ### END SOLUTION ###


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Define the forward pass of the DuelingQNetwork.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor (state).

        Returns
        -------
        torch.Tensor
            The output tensor (Q-values).
        """

        ### BEGIN SOLUTION ###
        
        ...

        q_values = ...

        ### END SOLUTION ###

        return q_values

#### Implement the training function

**Task 7.2:** complete the `train_dddqn_agent` function (you can simply copy / paste your response from `train_ddqn_agent`).

In [ ]:
def train_dddqn_agent(env: gym.Env,
                      q_network: torch.nn.Module,
                      target_q_network: torch.nn.Module,
                      optimizer: torch.optim.Optimizer,
                      loss_fn: Callable,
                      epsilon_greedy: EpsilonGreedy,
                      device: torch.device,
                      lr_scheduler: _LRScheduler,
                      num_episodes: int,
                      gamma: float,
                      batch_size: int,
                      replay_buffer: ReplayBuffer,
                      target_q_network_sync_period: int) -> List[float]:
    """
    Train the Q-network on the given environment.

    Parameters
    ----------
    env : gym.Env
        The environment to train on.
    q_network : torch.nn.Module
        The Q-network to train.
    target_q_network : torch.nn.Module
        The target Q-network to use for estimating the target Q-values.
    optimizer : torch.optim.Optimizer
        The optimizer to use for training.
    loss_fn : callable
        The loss function to use for training.
    epsilon_greedy : EpsilonGreedy
        The epsilon-greedy policy to use for action selection.
    device : torch.device
        The device to use for PyTorch computations.
    lr_scheduler : torch.optim.lr_scheduler._LRScheduler
        The learning rate scheduler to adjust the learning rate during training.
    num_episodes : int
        The number of episodes to train for.
    gamma : float
        The discount factor for future rewards.
    batch_size : int
        The size of the batch to use for training.
    replay_buffer : ReplayBuffer
        The replay buffer storing the experiences with their priorities.
    target_q_network_sync_period : int
        The number of episodes after which the target Q-network should be updated with the weights of the Q-network.

    Returns
    -------
    List[float]
        A list of cumulated rewards per episode.
    """
    iteration = 0
    episode_reward_list = []

    for episode_index in tqdm(range(1, num_episodes)):
        state, info = env.reset()
        episode_reward = 0

        for t in itertools.count():

            # GET ACTION, NEXT_STATE AND REWARD ###########

            action = epsilon_greedy(state)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            replay_buffer.add(state, action, reward, next_state, done)

            episode_reward += reward

            # UPDATE THE Q_NETWORK WEIGHTS WITH A BATCH OF EXPERIENCES FROM THE BUFFER

            if len(replay_buffer) > batch_size:
                batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones = replay_buffer.sample(batch_size)

                # Convert to PyTorch tensors
                batch_states_tensor = torch.tensor(batch_states, dtype=torch.float32, device=device)
                batch_actions_tensor = torch.tensor(batch_actions, dtype=torch.long, device=device)
                batch_rewards_tensor = torch.tensor(batch_rewards, dtype=torch.float32, device=device)
                batch_next_states_tensor = torch.tensor(batch_next_states, dtype=torch.float32, device=device)
                batch_dones_tensor = torch.tensor(batch_dones, dtype=torch.float32, device=device)

                # Compute the target Q values for the batch
                with torch.no_grad():

                    ### BEGIN SOLUTION ###

                    targets = ...

                    ### END SOLUTION ###

                # Compute the current Q values for the batch.
                # 
                # The expression `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)` is used to select specific elements from the tensor of Q-values returned by the Q-network.
                # 
                # Here's a breakdown of the following line of code:
                # - `q_network(batch_states_tensor)`:
                #   This is passing a batch of states through the Q-network.
                #   For each state, this outputs the Q-value for each possible action.
                #   Thus, `q_network(batch_states_tensor)` returns a tensor of shape (batch_size, action_dim).
                # 
                # - `gather(dim=1, index=batch_actions_tensor.unsqueeze(-1))`:
                #   This is selecting the Q-values corresponding to the actions that were actually taken.
                #   The `gather` function is used to select elements from a tensor using an index.
                #   In this case, the index is `batch_actions_tensor.unsqueeze(-1)`, which is a tensor of the actions that were taken.
                #   The `unsqueeze(-1)` function is used to add an extra dimension to the tensor, which is necessary for the `gather` function.
                # 
                # - `squeeze(-1)`:
                #   This is removing the extra dimension that was added by `unsqueeze(-1)`.
                #   The `squeeze` function is used to remove dimensions of size 1 from a tensor.
                #
                # So, the entire expression is selecting the Q-values of the actions that were actually taken from the tensor of all Q-values,
                # and returning a tensor of these selected Q-values.
                current_q_values = q_network(batch_states_tensor).gather(dim=1, index=batch_actions_tensor.unsqueeze(-1)).squeeze(-1)

                # Compute loss
                loss = loss_fn(current_q_values, targets)

                # Optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                lr_scheduler.step()

            # UPDATE THE TARGET Q-NETWORK #################

            # Every few training steps (e.g., every 100 steps), the weights of the target network are updated with the weights of the Q-network

            if iteration % target_q_network_sync_period == 0:
                target_q_network.load_state_dict(q_network.state_dict())

            iteration += 1

            if done:
                break

            state = next_state

        episode_reward_list.append(episode_reward)
        epsilon_greedy.decay_epsilon()

    return episode_reward_list

### Train it

In [ ]:
env = gym.make('CartPole-v1')

NUMBER_OF_TRAININGS = DEFAULT_NUMBER_OF_TRAININGS    # Change the default (global) value here if you want a specific number of trainings for this exercice
dddqn_trains_result_list = [[], [], []]

for train_index in range(NUMBER_OF_TRAININGS):

    # Instantiate required objects

    q_network = DuelingQNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device)
    target_q_network = DuelingQNetwork(state_dim, action_dim, nn_l1=128, nn_l2=128).to(device) # The target Q-network is used to compute the target Q-values for the loss function
    target_q_network.load_state_dict(q_network.state_dict()) # Initialize the target Q-network with the same weights as the Q-network

    optimizer = torch.optim.AdamW(q_network.parameters(), lr=0.004, amsgrad=True)
    #lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
    lr_scheduler = MinimumExponentialLR(optimizer, lr_decay=0.97, min_lr=0.0001)
    loss_fn = torch.nn.MSELoss()

    epsilon_greedy = EpsilonGreedy(epsilon_start=0.82, epsilon_min=0.013, epsilon_decay=0.9675, env=env, q_network=q_network)

    replay_buffer = ReplayBuffer(2000)

    # Train the q-network

    episode_reward_list = train_dddqn_agent(env,
                                            q_network,
                                            target_q_network,
                                            optimizer,
                                            loss_fn,
                                            epsilon_greedy,
                                            device,
                                            lr_scheduler,
                                            num_episodes=150,
                                            gamma=0.9,
                                            batch_size=128,
                                            replay_buffer=replay_buffer,
                                            target_q_network_sync_period=30)
    dddqn_trains_result_list[0].extend(range(len(episode_reward_list)))
    dddqn_trains_result_list[1].extend(episode_reward_list)
    dddqn_trains_result_list[2].extend([train_index for _ in episode_reward_list])

dddqn_trains_result_df = pd.DataFrame(np.array(dddqn_trains_result_list).T, columns=["num_episodes", "mean_final_episode_reward", "training_index"])
dddqn_trains_result_df["agent"] = "DDDQN"

# SAVE THE ACTION-VALUE ESTIMATION FUNCTION

torch.save(q_network, MODELS_DIR / "dddqn_q_network.pth")

env.close()

### Plot results

In [ ]:
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", estimator=None, units="training_index", data=dddqn_trains_result_df,
                height=7, aspect=2, alpha=0.5);

In [ ]:
#g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=dddqn_trains_result_df, height=7, aspect=2)

In [ ]:
all_trains_result_df = pd.concat([naive_trains_result_df, dqn1_trains_result_df, dqn2_trains_result_df, ddqn_trains_result_df, dqn_per_trains_result_df, dddqn_trains_result_df])
g = sns.relplot(x="num_episodes", y="mean_final_episode_reward", kind="line", hue="agent", data=all_trains_result_df, height=7, aspect=2)

### Test it

In [ ]:
# q_network = torch.load("dddqn_q_network.pth").to(device)

env = gym.make('CartPole-v1', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

test_q_network_agent(env, q_network, num_episode=3)

env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_dddqn_tained")

## Bonus 2: Hyperparameters optimization with Optuna

Optuna is an open-source hyperparameter optimization framework designed to automate the process of searching for the best hyperparameters in machine learning models. It is highly efficient and flexible, supporting various optimization algorithms. Optuna works with Python-based machine learning libraries like PyTorch, TensorFlow, and Scikit-learn. Optuna’s core feature is its ability to perform dynamic search spaces and pruning, allowing faster convergence by terminating poorly performing trials early.
Optuna supports distributed optimization for large-scale tuning.

### Official documentation

- Optuna GitHub: [https://github.com/optuna/optuna](https://github.com/optuna/optuna)
- Optuna Documentation: [https://optuna.org](https://optuna.org)

### Example of usage with PyTorch

Here's an example of how to use Optuna to optimize the hyperparameters of a simple neural network with PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset

# Define the PyTorch model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Objective function for Optuna
def objective(trial):
    # Hyperparameters to be tuned
    hidden_size = trial.suggest_int('hidden_size', 32, 128)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    
    model = Net(input_size=28*28, hidden_size=hidden_size, output_size=10)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # Dummy dataset
    X = torch.randn(100, 28*28)
    y = torch.randint(0, 10, (100,))
    train_loader = DataLoader(TensorDataset(X, y), batch_size=32)

    # Training loop
    for epoch in range(10):
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_x)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()

    return loss.item()

# Optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Show best hyperparameters
print(study.best_trial)

This example creates a basic neural network and tunes the `hidden_size` and learning rate (`lr`) using Optuna.

## Bonus 3: Monitoring the training process with *Weights & Biases*

Weights & Biases (W&B) is designed for machine learning practitioners to track and visualize experiments, making it an alternative to TensorBoard.
It integrates with frameworks like PyTorch, helping users monitor model training, compare hyperparameters, visualize performance metrics, and share results.

To get started, you'll need to [create a free account on the W&B website](https://wandb.ai/site). Once registered, a unique token will be generated to authenticate your account. This token is used with the `wandb.login()` function to log in and link your experiments to the W&B dashboard.

Below is a simple usage example, sourced from [this Colab notebook](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=1UU7kVQF2lOJ).

In [ ]:
import math
import wandb
import random

wandb.login()

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 1️. Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
          "learning_rate": 0.02,
          "architecture": "CNN",
          "dataset": "CIFAR-100",
          "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 2️. Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

You can visualize the training process in the Weights & Biases dashboard at https://wandb.ai/site.

To initialize a new W&B run, use `wandb.init()`:

```python
run = wandb.init(project="my-model-training-project")
```

You can log hyperparameters by setting `run.config = {...}`:

```python
run.config.update({"epochs": 1337, "learning_rate": 3e-4})
```

To track metrics, use the `run.log()` method:

```python
run.log({"metric": 42})
```

In addition, you can log artifacts such as models, datasets, and images:

```python
model_artifact = run.log_artifact("./model.pt", type="model")
```

## Bonus 4: Test and train a DQN agent to play Atari games

Training a DQN agent from scratch on the Atari environment can take several hours on a machine with a good GPU. Finding the right hyperparameters requires this training to be repeated many times. However, here you will find a pre-trained agents to test DQN on the Atari environment.

### Installation

Except if you have selected the `requirements-minimal.txt` file, atari is already installed. If not, you can install it with the following command:

- For Gymnasium  < 1.0.0: `pip install gymnasium[accept-rom-license,atari]`
- For Gymnasium >= 1.0.0: `pip install gymnasium[atari]`

As described in the following post `gymnasium[atari]` [is not compatible with Python 3.12](https://github.com/Farama-Foundation/Gymnasium/issues/1081) (the latest stable version of Python).
If you plan to complete this bonus section locally on your machine rather than on Google Colab, make sure you have Python 3.10 or 3.11 installed.

If you are using Python 3.12 and prefer not to run this notebook on Google Colab, an alternative solution via Docker is provided bellow.

#### Docker

We supose that you have Docker installed on your machine. If not, you can download it [here](https://www.docker.com/products/docker-desktop).

```bash
docker run -it --rm -v ./:/inf639 inf639:latest python3 ./your_dqn_training_script.py
```

If you want to monitor the training process with Weights & Biases, you can use the following command, assuming your token is stored in the `WANDB_API_KEY` environment variable:

```bash
docker run -it --rm -h your-hostname -e WANDB_API_KEY=$WANDB_API_KEY -e WANDB_PROJECT_NAME="inf639" -e WANDB_ENTITY="your-entity-name" -v ./:/inf639 inf639:latest python3 ./your_dqn_training_script.py
```

If you want to use a GPU within a Docker container, you have to install the [NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html) then you can use the following command:

```bash
docker run --gpus all -it --rm -h your-hostname -e WANDB_API_KEY=$WANDB_API_KEY -e WANDB_PROJECT_NAME="inf639" -e WANDB_ENTITY="your-entity-name" -v ./:/inf639 inf639:latest python3 ./your_dqn_training_script.py
```

If you have multiple GPUs, you can specify the GPU index with the `--gpus` option. For example, to use the second GPU, you can use the following command to run the training script on the first GPU:

```bash
docker run --gpus '"device=0"' -it --rm -h your-hostname -e WANDB_API_KEY=$WANDB_API_KEY -e WANDB_PROJECT_NAME="inf639" -e WANDB_ENTITY="your-entity-name" -v ./:/inf639 inf639:latest python3 ./your_dqn_training_script.py
```

and this one to run the training script on the second GPU:

```bash
docker run --gpus '"device=1"' -it --rm -h your-hostname -e WANDB_API_KEY=$WANDB_API_KEY -e WANDB_PROJECT_NAME="inf639" -e WANDB_ENTITY="your-entity-name" -v ./:/inf639 inf639:latest python3 ./your_dqn_training_script.py
```

### Hands on Atari environment

The Atari environment is a popular benchmark for reinforcement learning algorithms. It consists of a collection of over 50 Atari games, each with a unique set of challenges and rewards. The goal is to train an agent to play these games at a human-level performance or better.

The Atari environment is described [here](https://gymnasium.farama.org/environments/atari/).

In this lab, we will use the `ALE/Breakout-v5` environment, where the agent controls a paddle to bounce a ball and break a wall of bricks. The agent receives a reward for each brick it breaks and loses a life when the ball passes the paddle.

<img src="https://gymnasium.farama.org/_images/breakout.gif"></img>

This environment is described [here](https://gymnasium.farama.org/environments/atari/breakout/).

Print some information about the environment:

In [ ]:
env = gym.make('ALE/Breakout-v5', render_mode="rgb_array")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n.item()

print(f"State space size is: { env.observation_space }")
print(f"Action space size is: { env.action_space }")
print("Actions are: {" + ", ".join([str(a) for a in range(env.action_space.n)]) + "}")

env.close()

Run the following cells and check different basic 
policies (for instance randomly drawn actions) to discover the Breakout environment.

In [ ]:
env = gym.make('ALE/Breakout-v5', render_mode='rgb_array')
RenderWrapper.register(env, force_gif=True)

for episode_index in range(5):
    observation, info = env.reset()
    done = False

    for t in range(70):
        env.render_wrapper.render()

        if not done:
            print(observation)
        else:
            print("x", end="")

        action = env.action_space.sample()
        observation, reward, done, truncated, info = env.step(action)

    print()
    env.close()

env.render_wrapper.make_gif(FIGS_DIR / "lab1_ex8random")

### What is proposed in this lab

Training an Atari game model takes too long (around 4 hours with a recent GPU like an RTX 4060 or higher).
A pre-trained model has been optimized and reset only the last two layers (dense layers).
You now only need to retrain your model on these two specific layers.

This makes training much faster, reducing it to about ten minutes.

### Download pre-trained model

The pretrained model is available at the following URL: https://github.com/jeremiedecock/polytechnique-inf639-2024-students/raw/refs/heads/main/models/dqn_atari_weights_randomized_layers_7_and_9.pth

In [ ]:
!wget https://github.com/jeremiedecock/polytechnique-inf639-2024-students/raw/refs/heads/main/models/dqn_atari_weights_randomized_layers_7_and_9.pth

To load the pre-trained weights and freeze the convolutional layers, you can use the following code snippet:

```python
# ...

# Load pretrained weights if specified
q_network.load_state_dict(torch.load("dqn_atari_weights_randomized_layers_7_and_9.pth", map_location=device))

# Freeze convolutional layers
for param in q_network.network[:6].parameters():  # Assuming the first 6 layers are convolutional
    param.requires_grad = False

optimizer = optim.Adam(filter(lambda p: p.requires_grad, q_network.parameters()), lr=args.learning_rate)

# ...
```

### Model presentation

In the original DQN paper, the authors used a deep convolutional neural network to approximate the Q-function. The network consists of three convolutional layers followed by two fully connected layers. The input to the network is a stack of four consecutive frames, which allows the agent to perceive motion.
A ReLu activation function is used after each layer, except for the output layer, which uses a linear activation function.

If you need a refresher on convolutional neural networks, check out the following resources: 
- https://towardsdatascience.com/conv2d-to-finally-understand-what-happens-in-the-forward-pass-1bbaafb0b148
- https://github.com/AxelThevenot/GIF_convolutions

<img src="https://github.com/AxelThevenot/GIF_convolutions/blob/master/GIFS/Input%20Shape%20:%20(3,%207,%207)%20-%20Output%20Shape%20:%20(2,%207,%207)%20-%20K%20:%20(3,%203)%20-%20P%20:%20(1,%201)%20-%20S%20:%20(1,%201)%20-%20D%20:%20(1,%201)%20-%20G%20:%201.gif?raw=true" width=400></img>

### GPU

For CartPole, using a GPU was neither useful nor relevant because the model is too simple. The communication overhead between the CPU and GPU outweighed the speed gains in forward and backward pass computations.

However, for the Atari environment, the situation is different. The model is more complex, and using a GPU can significantly speed up the computations.

### Wrappers

The atari environment requires some preprocessing to be used with a DQN agent. The following wrappers are recommended:
- Stable Baselines3 Atari Wrappers: https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html
- Gymnasium Atari Wrappers: https://gymnasium.farama.org/api/wrappers/misc_wrappers/#gymnasium.wrappers.AtariPreprocessing

In the following implementation to complete, Stable Baselines3 Atari Wrappers are used.

[Machado paper](https://www.jair.org/index.php/jair/article/view/11182/26388) explains the importance of the preprocessing steps in the Atari environment.

### Network visualization with Zetane

You can use [Zetane](https://github.com/zetane/viewer?tab=readme-ov-file#zetane-viewer) to visualize the network architecture. It provides an interactive way to explore the structure of neural networks, including the layers, connections, and parameters.

### Hyperparameters

You can use the same hyperparameters than https://arxiv.org/abs/1312.5602 (DQN 2013) or https://arxiv.org/abs/1509.06461 (DDQN 2015) or https://arxiv.org/abs/1511.06581 (DDDQN 2015) or https://arxiv.org/abs/1511.05952 (PER 2016).

### Implement and train DQN on Breakout

Complete and test the following code to implement and train a DQN agent on the Breakout environment.

In [ ]:
import datetime
import gymnasium as gym
# import itertools
import numpy as np
import os
import pathlib
import random
from stable_baselines3.common.atari_wrappers import ClipRewardEnv, EpisodicLifeEnv, FireResetEnv, MaxAndSkipEnv, NoopResetEnv
from stable_baselines3.common.buffers import ReplayBuffer
import torch
# import tqdm

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

class QNetwork(torch.nn.Module):
    def __init__(self, env):
        super().__init__()

        self.network = torch.nn.Sequential(

        ### BEGIN SOLUTION ###
            
            ...

        ### END SOLUTION ###
        
        )


    def forward(self, x):
        return self.network(x / 255.0)


def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)


def make_env():
    #env = gym.make("ALE/Breakout-v5", render_mode="rgb_array")
    env = gym.make("BreakoutNoFrameskip-v4", render_mode="rgb_array")
    # https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    env = EpisodicLifeEnv(env)
    env = FireResetEnv(env)
    env = ClipRewardEnv(env)
    env = gym.wrappers.ResizeObservation(env, (84, 84))
    env = gym.wrappers.GrayScaleObservation(env)
    env = gym.wrappers.FrameStack(env, 4)
    return env


def train(
    num_training_timesteps: int = 3_000_000,
    learning_rate: float = 1e-4,
    buffer_size: int = 500_000,
    gamma: float = 0.99,
    batch_size: int = 32,
    initial_epsilon_value: float = 1.0,
    final_epsilon_value: float = 0.01,
    exploration_fraction: float = 0.10,
    initial_learning_timestep: int = 80_000,
    train_frequency: int = 4,
    target_network_frequency: int = 1_000,
    tau: float = 1.0,
    wandb_project_name: str = None,         # W&B's project name
    wandb_entity: str = None                # W&B's entity (username or team name)
):
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"dqn_atari_v2015_sb3_wrappers_{current_time}"

    print(run_name)

    # W&B setup
    if wandb_project_name is None:
        wandb_project_name = os.getenv('WANDB_PROJECT_NAME', None)

    if wandb_entity is None:
        wandb_entity = os.getenv('WANDB_ENTITY', None)

    use_wandb = wandb_project_name is not None and wandb_entity is not None

    if use_wandb:
        import wandb

        # wandb.login()

        wandb.init(
            project=wandb_project_name,
            entity=wandb_entity,
            # sync_tensorboard=True,
            config={
                "num_training_timesteps": num_training_timesteps,
                "learning_rate": learning_rate,
                "buffer_size": buffer_size,
                "gamma": gamma,
                "batch_size": batch_size,
                "initial_epsilon_value": initial_epsilon_value,
                "final_epsilon_value": final_epsilon_value,
                "exploration_fraction": exploration_fraction,
                "initial_learning_timestep": initial_learning_timestep,
                "train_frequency": train_frequency,
                "target_network_frequency": target_network_frequency,
                "tau": tau
            },
            name=run_name,
            monitor_gym=True,
            save_code=True,
        )

    # Create environment
    env = make_env()

    # Create networks
    q_network = QNetwork(env).to(DEVICE)

    # Load pretrained weights
    q_network.load_state_dict(torch.load("dqn_atari_weights_randomized_layers_7_and_9.pth", map_location=device))

    # Freeze convolutional layers
    for param in q_network.network[:6].parameters():  # Assuming the first 6 layers are convolutional
        param.requires_grad = False

    target_q_network = QNetwork(env).to(DEVICE)
    target_q_network.load_state_dict(q_network.state_dict())

    # Create optimizer and loss function
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, q_network.parameters()), lr=learning_rate)
    loss_fn = torch.nn.MSELoss()

    # Create replay buffer
    replay_buffer = ReplayBuffer(
        buffer_size,
        env.observation_space,
        env.action_space,
        DEVICE,
        optimize_memory_usage=True,
        handle_timeout_termination=False,
    )

    training_step = 0
    info = {"lives": 0}
    while training_step < num_training_timesteps:
        if info['lives'] == 0:
            observation, info = env.reset()
            episode_reward = 0

        done = False

        while not done:
            training_step += 1
            # env.render()

            # Take action with epsilon-greedy policy
            epsilon = linear_schedule(initial_epsilon_value, final_epsilon_value, exploration_fraction * num_training_timesteps, training_step)
            if random.random() < epsilon:
                action = np.array(env.action_space.sample())
            else:
                with torch.no_grad():
                    observation_batch_tensor = torch.Tensor(observation).unsqueeze(0).to(DEVICE)
                    q_values_tensor = q_network(observation_batch_tensor).squeeze(0)
                    action = torch.argmax(q_values_tensor).cpu().numpy()

            # Step environment
            next_observation, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward

            if done:
                if info['lives'] == 0:
                    print(f"Training step: {training_step} ; Episode reward: {episode_reward}")
                    # print(f"Lives: {info['lives']}")
                    if use_wandb:
                        wandb.log({
                            "training_step": training_step,
                            "episode_reward": episode_reward,
                            "num_transitions_in_replay_buffer": replay_buffer.size(),
                            "episode_frame_number": info["episode_frame_number"],
                            "frame_number": info["frame_number"],
                            "lives": info["lives"]
                        })

            # Add transition to replay buffer
            replay_buffer.add(observation, next_observation, action, reward, terminated, info)

            # Update the action value function
            if training_step > initial_learning_timestep:
                if training_step % train_frequency == 0:
                    # Sample a batch of transitions

                    ### BEGIN SOLUTION ###

                    td_target_batch = ...
                    q_value_batch = ...
                    loss = ...

                    ### END SOLUTION ###

                    if use_wandb and (training_step % (train_frequency * 10) == 0):
                        wandb.log({
                            "training_step": training_step,
                            "loss": loss,
                            "epsilon": epsilon,
                            "q_value_min":    q_value_batch.min().item(),
                            "q_value_max":    q_value_batch.max().item(),
                            "q_value_mean":   q_value_batch.mean().item(),
                            "q_value_median": q_value_batch.median().item(),
                            "td_target_min":    td_target_batch.min().item(),
                            "td_target_max":    td_target_batch.max().item(),
                            "td_target_mean":   td_target_batch.mean().item(),
                            "td_target_median": td_target_batch.median().item()
                        })

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                # Update target network
                if training_step % target_network_frequency == 0:
                    target_q_network.load_state_dict(q_network.state_dict())

            # Update the observation
            observation = next_observation
    
    # Save the model

    model_path = pathlib.Path("runs") / f"{run_name}_weights.pth"
    torch.save(q_network.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    env.close()

In [ ]:
train()

### Gradient clipping

Gradient clipping is a technique used to tackle the problem of exploding gradients in deep learning, including in the context of Reinforcement Learning. In reinforcement learning, the distribution of rewards and therefore gradients can be highly variable. Large updates to the Q-network weights can destabilize training, leading to divergence. Gradient clipping limits the size of the weight updates, ensuring stable and more reliable learning. By preventing erratic and large updates, gradient clipping can help the learning process converge more smoothly and often more quickly to a stable policy. DQN uses experience replay to break the correlation between successive updates. However, the mixed nature of the experiences can lead to high variance in gradients. Clipping gradients in this context ensures that even if there is a harmful experience in the replay buffer, it does not disproportionately affect the learning process.

Gradient clipping can be implemented using `torch.nn.utils.clip_grad_value_` between the `loss.backward()` and the `optimizer.step()` as follow:

```python
loss.backward()
torch.nn.utils.clip_grad_value_(q_network.parameters(), clip_grad_value)  # In-place gradient clipping
optimizer.step()
```

## Further readings

### Distributional methods

Distributional Q-Learning enhances the standard DQN by modeling the entire distribution of possible future rewards, rather than simply estimating the expected reward (https://arxiv.org/abs/1707.06887). The motivation for Distributional Q-Learning stems from the insight that the uncertainty in rewards—and consequently in the value function—can provide valuable information that is not captured when only the mean expected reward is considered.

Standard DQN approximates the expected value of the total return (the sum of future discounted rewards) from a given state-action pair. However, this approach ignores the variability around this expected value. In contrast, Distributional Q-Learning represents the value function as a distribution over possible returns, capturing the full range of potential outcomes and their probabilities.

This richer representation allows the agent to distinguish between actions that may lead to the same expected reward but with different risks or variances in outcomes. It enables more informed decision-making in stochastic environments, where the variability of returns is as important as the expectation. By capturing the entire distribution, Distributional Q-Learning can also potentially converge faster and yield more robust policies, as it accounts for the variance in rewards that can be critical in the learning process.

### NoisyNets DQN

Noisy DQN introduces parametric noise directly into the weights of the neural network to drive exploration, as opposed to traditional methods like ϵϵ-greedy where the randomness is injected into the action selection process (https://arxiv.org/abs/1706.10295). The key motivation behind Noisy DQN is to enable more sophisticated and efficient exploration strategies.

In standard DQN, exploration is often implemented through ϵϵ-greedy policies that select random actions with a certain probability ϵϵ, which can be suboptimal and inefficient, especially as ϵϵ has to be carefully decayed over time to balance exploration with exploitation. This randomness is external and does not adapt based on the agent’s experience.

Noisy DQN, however, incorporates noise into the network’s parameters, making the policy itself stochastic. This approach allows the network to learn the degree of exploration needed from the environment feedback, as the noise can be learned and adapted during training. It provides a more nuanced exploration mechanism that can potentially learn to explore in a state-dependent manner, leading to faster and more robust convergence of the learning process.

### Rainbow paper: Putting everything together

In  2017, Henssel et al. performed a large experiment that combined several DQN enhancements, among them DDQN and PER (https://arxiv.org/abs/1710.02298). They found that the ehancements worked well together. The paper has become known as the Rainbow paper, since the major graph showing the cumulative performance over 57 Atrari games is multicolored.

<img src="https://github.com/jeremiedecock/polytechnique-inf639-2024-students/blob/main/assets/lab1_rainbow_curve.png?raw=true" width="600px" />